In [ ]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from typing import Dict, List, Tuple, Optional, Union
import json
import os
import logging
from scipy.optimize import fsolve, brentq
import numpy_financial as npf

In [ ]:
# Step 1: Continental Extrapolation (Read Class SimpleRevenueAnalyzer for only processing raw deals, no extrapolation)
print("\n=== MODULE A ENHANCEMENT: CONTINENTAL EXTRAPOLATION ===")

class EnhancedSimpleRevenueAnalyzer:
    """
    ENHANCED SimpleRevenueAnalyzer with Continental Extrapolation
    
    Scales from 15 countries with actual data to ALL developing countries
    using hierarchical extrapolation: Direct → Continental → Global
    """
    
    def __init__(self, csv_path='renewable_energy_revenue_table_restructured_v9.csv', 
                 data_directory='.', carbon_arbitrage_path='./carbon_arbitrage_code'):
        
        print("="*80)
        print("EMDE RENEWABLE ENERGY ANALYSIS V12 WITH CONTINENTAL EXTRAPOLATION")
        print("="*80)
        
        self.csv_path = csv_path
        self.data_directory = data_directory
        self.carbon_arbitrage_path = carbon_arbitrage_path
        
        # Load datasets with encoding handling
        try:
            self.df_raw = pd.read_csv(csv_path, encoding='utf-8')
        except UnicodeDecodeError:
            try:
                self.df_raw = pd.read_csv(csv_path, encoding='latin-1')
            except:
                self.df_raw = pd.read_csv(csv_path, encoding='cp1252')
        print(f"Loaded {len(self.df_raw)} raw deals from {csv_path}")
        
        # Load classification data from carbon arbitrage codebase
        self.load_classification_data()
        
        # Filter to developing countries
        self.filter_to_developing_countries()
        
        # Analyze pricing patterns
        self.analyze_pricing_patterns()
        
        # Calculate country statistics (for the 15 countries with data)
        self.country_stats_df = self.calculate_country_statistics()
        
        # CONTINENTAL EXTRAPOLATION: Expand to all developing countries
        self.extrapolated_stats_df = self.extrapolate_to_all_countries()
        
        print(f"\\nCONTINENTAL EXTRAPOLATION COMPLETE:")
        print(f"Original countries with data: {len(self.country_stats_df)}")
        print(f"Total developing countries covered: {len(self.extrapolated_stats_df)}")
        print(f"Extrapolation ratio: {len(self.extrapolated_stats_df) / len(self.country_stats_df):.1f}x")
        
    def load_classification_data(self):
        """Load country classifications from carbon arbitrage codebase"""
        
        # Continental/regional mapping
        iso_path = os.path.join(self.carbon_arbitrage_path, 'data', 'country_ISO-3166_with_region.csv')
        self.iso_regions = pd.read_csv(iso_path)
        print(f"Loaded continental mapping for {len(self.iso_regions)} countries")
        
        # Development status
        unfccc_path = os.path.join(self.carbon_arbitrage_path, 'data', 'unfcc_classification_countries.csv')
        unfccc_df = pd.read_csv(unfccc_path, encoding='utf-8')
        
        # Clean the classification data
        unfccc_df = unfccc_df.dropna(subset=['asset_location', 'classification'])
        
        self.developing_countries = unfccc_df[
            unfccc_df['classification'] == 'Developing'
        ]['asset_location'].tolist()
        
        self.developed_countries = unfccc_df[
            unfccc_df['classification'] == 'All Developed'
        ]['asset_location'].tolist()
        
        print(f"Loaded development status: {len(self.developing_countries)} developing, {len(self.developed_countries)} developed")
        
        # Create country name mapping
        self.country_name_mapping = self._create_country_name_mapping()
        
    def _create_country_name_mapping(self) -> Dict[str, str]:
        """Create mapping from full country names to ISO2 codes"""
        mapping = {}
        
        # Manual mappings for countries that are already in dataset
        name_mappings = {
            'Argentina': 'AR', 'Bangladesh': 'BD', 'Brazil': 'BR', 'Chile': 'CL',
            'Egypt': 'EG', 'India': 'IN', 'Indonesia': 'ID', 'Kazakhstan': 'KZ',
            'Kenya': 'KE', 'Malaysia': 'MY', 'Mexico': 'MX', 'Morocco': 'MA',
            'Pakistan': 'PK', 'Peru': 'PE', 'Philippines': 'PH', 'Saudi Arabia': 'SA',
            'Senegal': 'SN', 'South Africa': 'ZA', 'Sri Lanka': 'LK', 'Thailand': 'TH',
            'USA': 'US', 'Zambia': 'ZM'
        }
        
        # Add ISO mappings
        for _, row in self.iso_regions.iterrows():
            mapping[row['name']] = row['alpha-2']
            
        # Override with manual mappings
        mapping.update(name_mappings)
        
        return mapping
    
    def filter_to_developing_countries(self):
        """Filter dataset to developing countries only"""
        
        # Map country names to ISO2 codes
        self.df_raw['country_iso2'] = self.df_raw['Country'].map(self.country_name_mapping)
        
        # Filter to developing countries only and clean data
        self.df_filtered = self.df_raw[
            self.df_raw['country_iso2'].isin(self.developing_countries)
        ].copy()
        
        # Clean revenue and capacity data - convert to numeric and filter valid data
        self.df_filtered['Revenue/MWh (USD 2023)'] = pd.to_numeric(
            self.df_filtered['Revenue/MWh (USD 2023)'], errors='coerce'
        )
        self.df_filtered['Capacity (MW)'] = pd.to_numeric(
            self.df_filtered['Capacity (MW)'], errors='coerce'
        )
        
        # Remove rows with invalid pricing or capacity data
        initial_count = len(self.df_filtered)
        self.df_filtered = self.df_filtered.dropna(subset=['Revenue/MWh (USD 2023)', 'Capacity (MW)'])
        self.df_filtered = self.df_filtered[
            (self.df_filtered['Revenue/MWh (USD 2023)'] > 0) & 
            (self.df_filtered['Capacity (MW)'] > 0)
        ]
        
        print(f"After filtering to developing countries: {len(self.df_filtered)} deals")
        print(f"Removed {initial_count - len(self.df_filtered)} deals with invalid pricing data")
        
        # Get unique countries and technologies
        unique_countries = sorted(self.df_filtered['Country'].unique())
        unique_technologies = sorted(self.df_filtered['Technology_Main'].unique())
        
        print(f"Countries found: {len(unique_countries)}")
        print(f"Countries: {unique_countries}")
        print(f"Technologies: {unique_technologies}")
        
    def analyze_pricing_patterns(self):
        """Analyze pricing patterns across technologies"""
        
        print(f"\\nTECHNOLOGY PRICE ANALYSIS:")
        tech_analysis = self.df_filtered.groupby('Technology_Main')['Revenue/MWh (USD 2023)'].agg(['mean', 'std', 'count'])
        print(tech_analysis)
        
        # Calculate technology averages for recommendations
        tech_prices = {}
        for tech in self.df_filtered['Technology_Main'].unique():
            tech_data = self.df_filtered[self.df_filtered['Technology_Main'] == tech]
            weighted_avg = np.average(tech_data['Revenue/MWh (USD 2023)'], weights=tech_data['Capacity (MW)'])
            tech_prices[tech] = weighted_avg
        
        print(f"\\nPricing Pattern Analysis:")
        print(f"Recommendation: SIMILAR PRICES ACROSS TECHNOLOGIES - Use single weighted average")
        for tech, price in sorted(tech_prices.items()):
            print(f"{tech}: ${price:.2f}/MWh")
            
    def calculate_country_statistics(self) -> pd.DataFrame:
        """Calculate statistics for countries with actual data"""
        
        country_stats = []
        
        for country in self.df_filtered['Country'].unique():
            country_data = self.df_filtered[self.df_filtered['Country'] == country]
            
            # Calculate weighted and unweighted averages
            weighted_avg = np.average(country_data['Revenue/MWh (USD 2023)'], weights=country_data['Capacity (MW)'])
            unweighted_avg = country_data['Revenue/MWh (USD 2023)'].mean()
            min_price = country_data['Revenue/MWh (USD 2023)'].min()
            max_price = country_data['Revenue/MWh (USD 2023)'].max()
            num_deals = len(country_data)
            
            country_stats.append({
                'Country': country,
                'country_iso2': self.country_name_mapping[country],
                'weighted_avg_usd_mwh': weighted_avg,
                'unweighted_avg_usd_mwh': unweighted_avg,
                'min_usd_mwh': min_price,
                'max_usd_mwh': max_price,
                'num_deals': num_deals,
                'data_source': 'direct'
            })
        
        country_stats_df = pd.DataFrame(country_stats)
        
        print(f"\\nCountry Statistics Summary:")
        print(f"Countries analyzed: {len(country_stats_df)}")
        print(f"Total deals: {len(self.df_filtered)}")
        print(f"Average deals per country: {len(self.df_filtered) / len(country_stats_df):.1f}")
        
        print(f"\\nCalculated statistics for {len(country_stats_df)} countries.")
        print("Country         weighted_avg_usd_mwh unweighted_avg_usd_mwh min_usd_mwh  max_usd_mwh  num_deals")
        for i, row in country_stats_df.iterrows():
            print(f"{i:2d} {row['Country']:<12} {row['weighted_avg_usd_mwh']:<18.2f} "
                  f"{row['unweighted_avg_usd_mwh']:<22.2f} {row['min_usd_mwh']:<12.2f} "
                  f"{row['max_usd_mwh']:<12.2f} {row['num_deals']}")
        
        return country_stats_df
    
    def extrapolate_to_all_countries(self) -> pd.DataFrame:
        """
        CONTINENTAL EXTRAPOLATION: Expand to all developing countries
        
        Hierarchy:
        1. Direct data (countries with actual deals)
        2. Continental average (same continent, developing countries only)
        3. Global developing average (fallback)
        """
        
        print(f"\\n" + "="*60)
        print("CONTINENTAL EXTRAPOLATION PROCESS")
        print("="*60)
        
        extrapolated_data = []
        
        # Get list of all developing countries from ISO data
        all_developing_iso2 = []
        for _, row in self.iso_regions.iterrows():
            iso2 = row['alpha-2']
            if iso2 in self.developing_countries:
                all_developing_iso2.append(iso2)
        
        print(f"Total developing countries to cover: {len(all_developing_iso2)}")
        
        # Statistics tracking
        direct_count = 0
        continental_count = 0
        global_count = 0
        
        # Calculate global averages for fallback
        global_weighted_avg = self.country_stats_df['weighted_avg_usd_mwh'].mean()
        global_min = self.country_stats_df['min_usd_mwh'].min()
        global_max = self.country_stats_df['max_usd_mwh'].max()
        
        for iso2 in all_developing_iso2:
            # Get country information
            country_info = self.iso_regions[self.iso_regions['alpha-2'] == iso2].iloc[0]
            country_name = country_info['name']
            continent = country_info['region']
            
            # Check if we have direct data
            direct_data = self.country_stats_df[self.country_stats_df['country_iso2'] == iso2]
            
            if len(direct_data) > 0:
                # USE DIRECT DATA
                row = direct_data.iloc[0]
                extrapolated_data.append({
                    'Country': row['Country'],
                    'country_iso2': iso2,
                    'country_name_iso': country_name,
                    'continent': continent,
                    'weighted_avg_usd_mwh': row['weighted_avg_usd_mwh'],
                    'min_usd_mwh': row['min_usd_mwh'],
                    'max_usd_mwh': row['max_usd_mwh'],
                    'data_source': 'direct',
                    'source_countries': row['Country']
                })
                direct_count += 1
                
            else:
                # Find countries in same continent with direct data
                continent_iso2_codes = self.iso_regions[
                    (self.iso_regions['region'] == continent) & 
                    (self.iso_regions['alpha-2'].isin(self.developing_countries))
                ]['alpha-2'].tolist()
                
                continent_countries_with_data = self.country_stats_df[
                    self.country_stats_df['country_iso2'].isin(continent_iso2_codes)
                ]
                
                if len(continent_countries_with_data) > 0:
                    # USE CONTINENTAL AVERAGE
                    continental_weighted_avg = continent_countries_with_data['weighted_avg_usd_mwh'].mean()
                    continental_min = continent_countries_with_data['min_usd_mwh'].min()
                    continental_max = continent_countries_with_data['max_usd_mwh'].max()
                    source_countries = ', '.join(continent_countries_with_data['Country'].tolist())
                    
                    extrapolated_data.append({
                        'Country': country_name,
                        'country_iso2': iso2,
                        'country_name_iso': country_name,
                        'continent': continent,
                        'weighted_avg_usd_mwh': continental_weighted_avg,
                        'min_usd_mwh': continental_min,
                        'max_usd_mwh': continental_max,
                        'data_source': 'continental',
                        'source_countries': source_countries
                    })
                    continental_count += 1
                    
                else:
                    # USE GLOBAL AVERAGE
                    source_countries = ', '.join(self.country_stats_df['Country'].tolist())
                    
                    extrapolated_data.append({
                        'Country': country_name,
                        'country_iso2': iso2,
                        'country_name_iso': country_name,
                        'continent': continent,
                        'weighted_avg_usd_mwh': global_weighted_avg,
                        'min_usd_mwh': global_min,
                        'max_usd_mwh': global_max,
                        'data_source': 'global',
                        'source_countries': source_countries
                    })
                    global_count += 1
        
        extrapolated_df = pd.DataFrame(extrapolated_data)
        
        print(f"\\nEXTRAPOLATION RESULTS:")
        print(f"Direct data: {direct_count} countries")
        print(f"Continental extrapolation: {continental_count} countries")
        print(f"Global average fallback: {global_count} countries")
        print(f"Total coverage: {len(extrapolated_df)} developing countries")
        
        # Show sample of extrapolated countries by continent
        print(f"\\nSAMPLE EXTRAPOLATED COUNTRIES BY CONTINENT:")
        for continent in extrapolated_df['continent'].unique():
            continent_data = extrapolated_df[extrapolated_df['continent'] == continent]
            continental_only = continent_data[continent_data['data_source'] == 'continental']
            if len(continental_only) > 0:
                sample = continental_only.head(3)
                print(f"{continent}: {len(continental_only)} countries, e.g., {', '.join(sample['Country'].tolist())}")
        
        # Export for DCF analysis
        self.export_dcf_compatible_data(extrapolated_df)
        
        return extrapolated_df
    
    def export_dcf_compatible_data(self, extrapolated_df: pd.DataFrame):
        """Export extrapolated data in DCF-compatible format"""
        
        export_data = []
        
        for _, row in extrapolated_df.iterrows():
            export_data.append({
                'country_name': row['Country'],
                'country_iso2': row['country_iso2'],
                'continent': row['continent'],
                'weighted_avg_usd_mwh': row['weighted_avg_usd_mwh'],
                'min_usd_mwh': row['min_usd_mwh'],
                'max_usd_mwh': row['max_usd_mwh'],
                'data_source': row['data_source'],
                'source_countries': row['source_countries']
            })
        
        export_df = pd.DataFrame(export_data)
        export_path = 'revenue_analysis_continental_extrapolation_v12.csv'
        export_df.to_csv(export_path, index=False)
        
        print(f"\\nExported continental extrapolation data: {export_path}")
        print(f"Countries: {len(export_df)}")
        print(f"Ready for DCF analysis across all developing countries")
        
        return export_path

analyzer = EnhancedSimpleRevenueAnalyzer()
extrapolated_df = analyzer.extrapolate_to_all_countries()

In [ ]:
"""
EMDE Renewable Energy Revenue Analysis V9
Revenue analysis for EMDE renewable energy projects.
"""

class SimpleRevenueAnalyzer:
    """    
    Data Features:
    - Real renewable deals post-2015 analysis
    - Technology-specific pricing analysis
    - Country-level statistical summaries
    - Weighted average, minimum, and maximum methodologies
    """
    
    def __init__(self):
        self.revenue_data = None
        self.technology_colors = {
            'Solar': '#FFA500',           # Orange
            'Wind Onshore': '#32CD32',    # Green  
            'Wind Offshore': '#006400',   # Dark Green
            'Hydropower': '#1E90FF',      # Blue
            'Geothermal': '#8B4513',      # Brown
            'Other': '#808080'            # Gray
        }
        
        # Technology standardisation mapping
        self.technology_mapping = {
            'solar': 'Solar',
            'Solar PV': 'Solar',
            'Solar': 'Solar', 
            'Solar photovoltaic': 'Solar',
            'PV': 'Solar',
            'Photovoltaic': 'Solar',
            'onshore_wind': 'Wind Onshore',
            'Wind - Onshore': 'Wind Onshore',
            'Wind Onshore': 'Wind Onshore',
            'Wind onshore': 'Wind Onshore', 
            'Onshore Wind': 'Wind Onshore',
            'Wind': 'Wind Onshore',
            'offshore_wind': 'Wind Offshore',
            'Wind - Offshore': 'Wind Offshore',
            'Wind Offshore': 'Wind Offshore',
            'Wind offshore': 'Wind Offshore',
            'Offshore Wind': 'Wind Offshore',
            'hydropower': 'Hydropower',
            'Hydroelectric': 'Hydropower',
            'Hydro': 'Hydropower',
            'Hydropower': 'Hydropower',
            'Hydroelectric power': 'Hydropower',
            'Small Hydro': 'Hydropower',
            'Large Hydro': 'Hydropower',
            'geothermal': 'Geothermal',
            'Geothermal': 'Geothermal',
            'Geothermal power': 'Geothermal'
        }

    def _standardize_technologies(self, tech_series: pd.Series) -> pd.Series:
        """Standardize technology names using mapping."""
        def safe_map(x):
            if pd.isna(x):
                return 'Other'
            x_clean = str(x).strip()
            
            # Try exact match first
            if x_clean in self.technology_mapping:
                return self.technology_mapping[x_clean]
            
            # Try case-sensitive match
            for key, value in self.technology_mapping.items():
                if x_clean.lower() == key.lower():
                    return value
            
            # Check for key terms
            x_lower = x_clean.lower()
            if 'solar' in x_lower or 'pv' in x_lower:
                return 'Solar'
            elif 'wind' in x_lower:
                if 'offshore' in x_lower or 'off-shore' in x_lower:
                    return 'Wind Offshore'
                else:
                    return 'Wind Onshore'
            elif 'hydro' in x_lower:
                return 'Hydropower'
            elif 'geothermal' in x_lower:
                return 'Geothermal'
            else:
                return 'Other'
        
        return tech_series.apply(safe_map)

    def load_and_clean_revenue_data(self, csv_path: str = 'renewable_energy_revenue_table_restructured_v9.csv') -> pd.DataFrame:
        """
        Load and clean revenue data.
        Complemented with data validation.
        
        Returns:
        --------
        pd.DataFrame: Cleaned revenue data with ALL countries preserved
        """
        try:
            if not Path(csv_path).exists():
                raise FileNotFoundError(f"Revenue data file not found: {csv_path}")
            
            # Load with encoding handling
            try:
                df = pd.read_csv(csv_path, encoding='cp1252')
            except UnicodeDecodeError:
                df = pd.read_csv(csv_path, encoding='utf-8')
            
            print(f"Loaded {len(df)} raw deals from {csv_path}")
            
            # Initial data validation and cleaning
            df = df.dropna(subset=['Revenue/MWh (USD 2023)', 'Capacity (MW)'])
            df['Revenue_USD'] = pd.to_numeric(df['Revenue/MWh (USD 2023)'], errors='coerce')
            df['Capacity_MW'] = pd.to_numeric(df['Capacity (MW)'], errors='coerce')
            
            # Remove outliers using (intuitively) reasonable thresholds
            df = df[(df['Revenue_USD'] > 0) & (df['Revenue_USD'] < 500)]
            df = df[(df['Capacity_MW'] > 0) & (df['Capacity_MW'] < 5000)]
            
            # Technology standardization
            df['Technology_Clean'] = self._standardize_technologies(df['Technology_Main'])
            
            # Filter to developing countries only 
            developing_data = df[df['UNFCCC_Classification'] == 'Developing'].copy()
            
            if developing_data.empty:
                raise ValueError("No developing country data found after filtering")
            
            # Print debugging information
            print(f"After filtering to developing countries: {len(developing_data)} deals")
            print(f"Countries found: {developing_data['Country'].nunique()}")
            print(f"Countries: {sorted(developing_data['Country'].unique())}")
            print(f"Technologies: {sorted(developing_data['Technology_Clean'].unique())}")
            
            return developing_data
            
        except Exception as e:
            print(f"CRITICAL ERROR loading revenue data: {e}")
            raise

    def analyze_pricing_patterns(self, df: pd.DataFrame) -> Tuple[str, Dict[str, float]]:
        """
        Analyze pricing patterns to determine weighting methodology.
        
        Parameters:
        -----------
        df : pd.DataFrame
            Cleaned revenue data
            
        Returns:
        --------
        Tuple[str, Dict[str, float]]: Recommendation and technology prices
        """
        try:
            print("\nTECHNOLOGY PRICE ANALYSIS:")
            # Calculate statistics by technology
            tech_stats = df.groupby('Technology_Clean')['Revenue_USD'].agg(['mean', 'std', 'count'])
            print(tech_stats)
            
            # Calculate weighted average prices by technology
            tech_weighted_prices = {}
            for tech in df['Technology_Clean'].unique():
                tech_data = df[df['Technology_Clean'] == tech]
                if len(tech_data) > 0:
                    weighted_price = np.average(tech_data['Revenue_USD'], 
                                              weights=tech_data['Capacity_MW'])
                    tech_weighted_prices[tech] = round(weighted_price, 2)
            
            # Analyze price variance
            price_values = list(tech_weighted_prices.values())
            if len(price_values) > 1:
                price_cv = np.std(price_values) / np.mean(price_values)
                
                if price_cv < 0.3:
                    recommendation = "SIMILAR PRICES ACROSS TECHNOLOGIES - Use single weighted average"
                else:
                    recommendation = "SIGNIFICANT PRICE VARIATION - Use technology-specific pricing"
            else:
                recommendation = "INSUFFICIENT DATA - Use weighted average"
            
            print(f"\nPricing Pattern Analysis:")
            print(f"Recommendation: {recommendation}")
            for tech, price in tech_weighted_prices.items():
                print(f"{tech}: ${price:.2f}/MWh")
            
            return recommendation, tech_weighted_prices
            
        except Exception as e:
            print(f"Error in pricing pattern analysis: {e}")
            return "ERROR - Use weighted average", {}

    def calculate_country_statistics(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Calculate comprehensive country-level pricing statistics.
                
        Parameters:
        -----------
        df : pd.DataFrame
            Cleaned revenue data
            
        Returns:
        --------
        pd.DataFrame: Country statistics with weighted avg, unweighted avg, min, max, num_deals
        """
        try:
            country_stats = []
            
            for country in sorted(df['Country'].unique()):
                country_data = df[df['Country'] == country].copy()
                
                if len(country_data) == 0:
                    continue
                
                # Calculate all crit statistics
                revenues = country_data['Revenue_USD'].values
                capacities = country_data['Capacity_MW'].values
                
                # Weighted average (capacity-weighted)
                total_capacity = capacities.sum()
                weighted_avg = np.average(revenues, weights=capacities)
                
                # Unweighted average
                unweighted_avg = revenues.mean()
                
                # Min and Max
                min_price = revenues.min()
                max_price = revenues.max()
                
                # Count
                num_deals = len(country_data)
                
                # Additional metadata
                total_capacity_mw = total_capacity
                tech_count = country_data['Technology_Clean'].nunique()
                
                # Primary technology by capacity
                tech_capacity = country_data.groupby('Technology_Clean')['Capacity_MW'].sum()
                primary_tech = tech_capacity.idxmax()
                
                country_stats.append({
                    'Country': country,
                    'weighted_avg_usd_mwh': round(weighted_avg, 2),
                    'unweighted_avg_usd_mwh': round(unweighted_avg, 2),
                    'min_usd_mwh': round(min_price, 2),
                    'max_usd_mwh': round(max_price, 2),
                    'num_deals': num_deals,
                    'total_capacity_mw': round(total_capacity_mw, 2),
                    'technology_count': tech_count,
                    'primary_technology': primary_tech
                })
            
            return pd.DataFrame(country_stats)
            
        except Exception as e:
            print(f"Error calculating country statistics: {e}")
            return pd.DataFrame()

    def create_revenue_visualization(self, df: pd.DataFrame) -> Tuple[plt.Figure, List[str]]:
        """
        Create comprehensive revenue analysis visualizations.
        Allows for comparative analysis across EMDEs.
        """
        try:
            # Configurate za plot style
            plt.style.use('default')
            sns.set_palette("husl")
            
            fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))
            fig.suptitle('EMDE Renewable Energy Revenue Analysis V9\nDeal Pricing by Country and Technology',
                        fontsize=16, fontweight='bold')
            
            # Get country statistics for sorting
            country_stats = self.calculate_country_statistics(df)
            countries = country_stats['Country'].tolist()
            
            # Plot 1: Deal prices by country and technology (scatter)
            for i, tech in enumerate(sorted(df['Technology_Clean'].unique())):
                if tech in self.technology_colors:
                    tech_data = df[df['Technology_Clean'] == tech]
                    # Map countries to x-axis positions
                    x_positions = [countries.index(country) for country in tech_data['Country'] if country in countries]
                    y_values = tech_data[tech_data['Country'].isin(countries)]['Revenue_USD'].values[:len(x_positions)]
                    
                    ax1.scatter(x_positions, y_values,
                              color=self.technology_colors[tech], label=tech, alpha=0.7, s=60)
            
            ax1.set_title('Deal Pricing by Country and Technology', fontweight='bold')
            ax1.set_ylabel('Revenue (USD/MWh)')
            ax1.set_xlabel('Countries')
            ax1.set_xticks(range(len(countries)))
            ax1.set_xticklabels([c[:3] for c in countries], rotation=45)
            ax1.legend()
            ax1.grid(True, alpha=0.4)
            
            # Plot 2: Three methodologies comparison (SORTED BY WEIGHTED AVERAGE)
            # Sort countries by weighted average for this plot
            sorted_stats = country_stats.sort_values('weighted_avg_usd_mwh').reset_index(drop=True)
            
            x_positions = range(len(sorted_stats))
            ax2.plot(x_positions, sorted_stats['weighted_avg_usd_mwh'], 
                    color='blue', marker='o', linewidth=2, label='Weighted Avg')
            ax2.plot(x_positions, sorted_stats['min_usd_mwh'], 
                    color='green', marker='o', linewidth=2, label='Minimum')  
            ax2.plot(x_positions, sorted_stats['max_usd_mwh'], 
                    color='red', marker='o', linewidth=2, label='Maximum')
            
            ax2.set_title('Three Pricing Methodologies Comparison', fontweight='bold')
            ax2.set_ylabel('Revenue (USD/MWh)')
            ax2.set_xlabel('Countries (Sorted by Weighted Average)')
            ax2.legend()
            ax2.grid(True, alpha=0.3)
            
            # Plot 3: Price variability by country
            country_stats['price_range'] = country_stats['max_usd_mwh'] - country_stats['min_usd_mwh']
            country_stats['coefficient_variation'] = (country_stats['price_range'] / 
                                                    country_stats['weighted_avg_usd_mwh'] * 100)
            
            bars = ax3.bar(range(len(country_stats)), country_stats['coefficient_variation'],
                          color='lightblue', alpha=0.7)
            ax3.set_title('Price Variability by Country', fontweight='bold')
            ax3.set_ylabel('Coefficient of Variation (%)')
            ax3.set_xlabel('Countries')
            ax3.set_xticks(range(len(country_stats)))
            ax3.set_xticklabels([c[:3] for c in country_stats['Country']], rotation=45, fontsize=8)
            
            # Plot 4: Technology mix by country
            tech_mix_data = []
            for country in countries:
                country_data = df[df['Country'] == country]
                tech_counts = country_data['Technology_Clean'].value_counts()
                total_deals = len(country_data)
                
                tech_mix = {}
                for tech in self.technology_colors.keys():
                    tech_mix[tech] = (tech_counts.get(tech, 0) / total_deals) * 100
                tech_mix['country'] = country
                tech_mix_data.append(tech_mix)
            
            tech_mix_df = pd.DataFrame(tech_mix_data)
            
            bottom = np.zeros(len(tech_mix_df))
            for tech in self.technology_colors.keys():
                if tech in tech_mix_df.columns:
                    ax4.bar(range(len(tech_mix_df)), tech_mix_df[tech], 
                           bottom=bottom, label=tech, color=self.technology_colors[tech], alpha=0.8)
                    bottom += tech_mix_df[tech].fillna(0)
            
            ax4.set_title('Technology Mix by Country', fontweight='bold')
            ax4.set_ylabel('Share of Deals (%)')
            ax4.set_xlabel('Countries')
            ax4.set_xticks(range(len(tech_mix_df)))
            ax4.set_xticklabels([c[:3] for c in countries], rotation=45, fontsize=8)
            ax4.legend()
            
            plt.tight_layout()
            
            return fig, countries
            
        except Exception as e:
            print(f"Error creating visualizations: {e}")
            return plt.figure(), []

    def export_dcf_compatible_data(self, country_stats: pd.DataFrame, df: pd.DataFrame) -> Tuple[pd.DataFrame, str]:
        """
        Export data in DCF-compatible format for Module A integration.
        """
        try:
            # Create DCF-compatible export with all three methodologies
            dcf_export = []
            
            for _, country_row in country_stats.iterrows():
                country = country_row['Country']
                
                # Weighted average methodology
                dcf_export.append({
                    'Country': country,
                    'pricing_methodology': 'weighted_avg',
                    'revenue_per_mwh': country_row['weighted_avg_usd_mwh'],
                    'total_capacity_mw': country_row['total_capacity_mw'],
                    'deal_count': country_row['num_deals'],
                    'technology_count': country_row['technology_count'],
                    'primary_technology': country_row['primary_technology'],
                    'data_source': 'Simple Revenue Analyzer V9',
                    'export_date': pd.Timestamp.now().strftime('%Y-%m-%d')
                })
                
                # Minimum methodology  
                dcf_export.append({
                    'Country': country,
                    'pricing_methodology': 'minimum',
                    'revenue_per_mwh': country_row['min_usd_mwh'],
                    'total_capacity_mw': country_row['total_capacity_mw'],
                    'deal_count': country_row['num_deals'],
                    'technology_count': country_row['technology_count'],
                    'primary_technology': country_row['primary_technology'],
                    'data_source': 'Simple Revenue Analyzer V9',
                    'export_date': pd.Timestamp.now().strftime('%Y-%m-%d')
                })
                
                # Maximum methodology
                dcf_export.append({
                    'Country': country,
                    'pricing_methodology': 'maximum',
                    'revenue_per_mwh': country_row['max_usd_mwh'],
                    'total_capacity_mw': country_row['total_capacity_mw'],
                    'deal_count': country_row['num_deals'],
                    'technology_count': country_row['technology_count'],
                    'primary_technology': country_row['primary_technology'],
                    'data_source': 'Simple Revenue Analyzer V9',
                    'export_date': pd.Timestamp.now().strftime('%Y-%m-%d')
                })
            
            dcf_df = pd.DataFrame(dcf_export)
            
            # Export to CSV
            filename = 'revenue_analysis_for_dcf_v9.csv'
            dcf_df.to_csv(filename, index=False)
            
            print(f"\nExported DCF-compatible data: {filename}")
            print(f"Countries: {country_stats['Country'].nunique()}")
            print(f"Total methodologies: {len(dcf_df)} (3 per country)")
            
            return dcf_df, filename
            
        except Exception as e:
            print(f"Error exporting DCF data: {e}")
            return pd.DataFrame(), ""

    def run_complete_revenue_analysis(self, csv_path: str = None) -> Tuple[pd.DataFrame, Dict]:
        """
        Execute complete revenue analysis pipeline.
        """
        print("=" * 80)
        print("EMDE RENEWABLE ENERGY REVENUE ANALYSIS V10")
        print("=" * 80)
        
        # Load and clean data
        if csv_path is None:
            csv_path = 'renewable_energy_revenue_table_restructured_v9.csv'
        
        df = self.load_and_clean_revenue_data(csv_path)
        
        # Analyze pricing patterns
        recommendation, tech_prices = self.analyze_pricing_patterns(df)
        
        # Calculate country statistics
        country_stats = self.calculate_country_statistics(df)
        
        print(f"\nCountry Statistics Summary:")
        print(f"Countries analyzed: {country_stats['Country'].nunique()}")
        print(f"Total deals: {len(df)}")
        print(f"Average deals per country: {len(df) / country_stats['Country'].nunique():.1f}")
        
        # Display statistical details as table
        print(f"\nCalculated statistics for {len(country_stats)} countries.")
        print("Country".ljust(15), "weighted_avg_usd_mwh".ljust(20), "unweighted_avg_usd_mwh".ljust(22), 
              "min_usd_mwh".ljust(12), "max_usd_mwh".ljust(12), "num_deals")
        
        for index, row in country_stats.iterrows():
            print(f"{index:2d} {row['Country']:<13} {row['weighted_avg_usd_mwh']:<18} "
                  f"{row['unweighted_avg_usd_mwh']:<20} {row['min_usd_mwh']:<10} "
                  f"{row['max_usd_mwh']:<10} {row['num_deals']}")
        
        # Generate visuals
        fig, countries_with_data = self.create_revenue_visualization(df)
        
        # Export DCF-compatible data
        dcf_data, dcf_filename = self.export_dcf_compatible_data(country_stats, df)
        
        analysis_results = {
            'recommendation': recommendation,
            'technology_prices': tech_prices,
            'countries_analyzed': countries_with_data,
            'dcf_filename': dcf_filename,
            'visualization': fig,
            'raw_data': df,
            'methodologies_count': len(dcf_data),
            'country_stats': country_stats
        }
        
        print(f"\nAnalysis complete. Data exported to: {dcf_filename}")
        print("Ready for DCF Module A integration.")
        
        # EXPLANATION OF METHODOLOGY
        print("\n" + "=" * 60)
        print("METHODOLOGY EXPLANATION:")
        print("=" * 60)
        print("'Countries (Sorted)' X-Axis: Countries are sorted by their pricing")
        print("methodology values to create visual rankings and identify trends.")
        print("=" * 60)
        
        return country_stats, analysis_results

# Example usage function for standalone testing
def run_simple_revenue_analysis():
    """Standalone execution function for testing."""
    analyzer = SimpleRevenueAnalyzer()
    stats, results = analyzer.run_complete_revenue_analysis()
    
    # Display key results
    print(f"\nKey Results:")
    print(f"- Pricing recommendation: {results['recommendation']}")
    print(f"- Countries with data: {len(results['countries_analyzed'])}")
    print(f"- DCF export file: {results['dcf_filename']}")
    print(f"- Total methodologies: {results['methodologies_count']}")
    
    return analyzer, stats, results

if __name__ == "__main__":
    analyzer, stats, results = run_simple_revenue_analysis()
    plt.show()

In [ ]:
"""
Module A - DCF/NPV Analysis
EMDE Renewable Energy Financial Analysis
FIXES:
1. Complete country code mapping for all 154 countries
2. Opportunity cost integration (only until 2050)
3. Fixed CSV column names for V12 extrapolation data
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from pathlib import Path
from add_opportunity_costs import OpportunityCostIntegrator, add_opportunity_costs_to_dcf_cash_flows

# Configuration Constants 
DISCOUNT_RATE = 0.02795381840850683 # Magic Number from PUB's 
HOURS_IN_YEAR = 8760
TERMINAL_GROWTH_RATE = 0.02

# Renewable lifespans (years) - UNIT BASIS FROM PUB's
RENEWABLE_LIFESPAN = {
    'solar': 30,        # years (everything except batteries is 30 years)
    'onshore_wind': 30,
    'offshore_wind': 30,
    'hydropower': 30,
    'geothermal': 30
}

# Energy type mapping to CSV columns
ENERGY_MAPPING = {
    'solar': 'Solar',
    'onshore_wind': 'Wind_Onshore',
    'offshore_wind': 'Wind_Offshore',
    'hydropower': 'Hydropower',
    'geothermal': 'Geothermal'
}

# COMPLETE COUNTRY CODE MAPPING FOR ALL 154 COUNTRIES
COMPLETE_COUNTRY_CODES = {
    'Argentina': 'AR', 'Brazil': 'BR', 'Chile': 'CL', 'India': 'IN', 'Indonesia': 'ID',
    'Bangladesh': 'BD', 'Philippines': 'PH', 'Thailand': 'TH', 'Malaysia': 'MY',
    'South Africa': 'ZA', 'Egypt': 'EG', 'Morocco': 'MA', 'Kenya': 'KE',
    'Kazakhstan': 'KZ', 'Pakistan': 'PK', 'Peru': 'PE', 'Colombia': 'CO',
    'China': 'CN', 'Mexico': 'MX', 'Turkey': 'TR', 'UAE': 'AE',
    'Myanmar': 'MM', 'Nigeria': 'NG', 'Senegal': 'SN', 'Sri Lanka': 'LK',
    'Taiwan': 'TW', 'Zambia': 'ZM', 'Saudi Arabia': 'SA',
    'Afghanistan': 'AF', 'Albania': 'AL', 'Algeria': 'DZ', 'Andorra': 'AD', 
    'Angola': 'AO', 'Antigua and Barbuda': 'AG', 'Armenia': 'AM', 'Azerbaijan': 'AZ',
    'Bahamas': 'BS', 'Bahrain': 'BH', 'Barbados': 'BB', 'Belize': 'BZ', 'Benin': 'BJ',
    'Bhutan': 'BT', 'Bolivia (Plurinational State of)': 'BO', 'Bosnia and Herzegovina': 'BA',
    'Botswana': 'BW', 'Brunei Darussalam': 'BN', 'Burkina Faso': 'BF', 'Burundi': 'BI',
    'Cabo Verde': 'CV', 'Cambodia': 'KH', 'Cameroon': 'CM', 'Central African Republic': 'CF',
    'Chad': 'TD', 'Comoros': 'KM', 'Congo': 'CG', 'Congo, Democratic Republic of the': 'CD',
    'Cook Islands': 'CK', 'Costa Rica': 'CR', 'Côte d\'Ivoire': 'CI', 'Cuba': 'CU',
    'Djibouti': 'DJ', 'Dominica': 'DM', 'Dominican Republic': 'DO', 'Ecuador': 'EC',
    'El Salvador': 'SV', 'Equatorial Guinea': 'GQ', 'Eritrea': 'ER', 'Eswatini': 'SZ',
    'Ethiopia': 'ET', 'Fiji': 'FJ', 'Gabon': 'GA', 'Gambia': 'GM', 'Georgia': 'GE',
    'Ghana': 'GH', 'Grenada': 'GD', 'Guatemala': 'GT', 'Guinea': 'GN', 'Guinea-Bissau': 'GW',
    'Guyana': 'GY', 'Haiti': 'HT', 'Holy See': 'VA', 'Honduras': 'HN', 'Iran (Islamic Republic of)': 'IR',
    'Iraq': 'IQ', 'Israel': 'IL', 'Jamaica': 'JM', 'Jordan': 'JO', 'Kiribati': 'KI',
    'Korea (Democratic People\'s Republic of)': 'KP', 'Korea, Republic of': 'KR', 'Kuwait': 'KW',
    'Kyrgyzstan': 'KG', 'Lao People\'s Democratic Republic': 'LA', 'Lebanon': 'LB', 'Lesotho': 'LS',
    'Liberia': 'LR', 'Libya': 'LY', 'Madagascar': 'MG', 'Malawi': 'MW', 'Maldives': 'MV',
    'Mali': 'ML', 'Marshall Islands': 'MH', 'Mauritania': 'MR', 'Mauritius': 'MU',
    'Micronesia (Federated States of)': 'FM', 'Moldova, Republic of': 'MD', 'Mongolia': 'MN',
    'Montenegro': 'ME', 'Mozambique': 'MZ', 'Nauru': 'NR', 'Nepal': 'NP', 'Nicaragua': 'NI',
    'Niger': 'NE', 'Niue': 'NU', 'North Macedonia': 'MK', 'Oman': 'OM', 'Palau': 'PW',
    'Palestine, State of': 'PS', 'Panama': 'PA', 'Papua New Guinea': 'PG', 'Paraguay': 'PY',
    'Qatar': 'QA', 'Rwanda': 'RW', 'Saint Kitts and Nevis': 'KN', 'Saint Lucia': 'LC',
    'Saint Vincent and the Grenadines': 'VC', 'Samoa': 'WS', 'San Marino': 'SM',
    'Sao Tome and Principe': 'ST', 'Serbia': 'RS', 'Seychelles': 'SC', 'Sierra Leone': 'SL',
    'Singapore': 'SG', 'Solomon Islands': 'SB', 'Somalia': 'SO', 'South Sudan': 'SS',
    'Sudan': 'SD', 'Suriname': 'SR', 'Syrian Arab Republic': 'SY', 'Tajikistan': 'TJ',
    'Tanzania, United Republic of': 'TZ', 'Timor-Leste': 'TL', 'Togo': 'TG', 'Tonga': 'TO',
    'Trinidad and Tobago': 'TT', 'Tunisia': 'TN', 'Turkmenistan': 'TM', 'Tuvalu': 'TV',
    'Uganda': 'UG', 'United Arab Emirates': 'AE', 'Uruguay': 'UY', 'Uzbekistan': 'UZ',
    'Vanuatu': 'VU', 'Venezuela (Bolivarian Republic of)': 'VE', 'Viet Nam': 'VN', 'Yemen': 'YE',
    'Zimbabwe': 'ZW'
}

class RenewableEnergyDCFV9:
    """
    V9 = V8 + SimpleRevenueAnalyzer Integration + Three Pricing Options + Opportunity Costs
    - Uses revenue_analysis_continental_extrapolation_v12.csv 
    - Supports three pricing methodologies: weighted_avg, minimum, maximum
    - Integrates PUB's opportunity costs (phase-out costs)
    """
    
    def __init__(self, country_name, country_code, pricing_methodology='weighted_avg', data_directory='.'):
        self.country_name = country_name
        self.country_code = country_code
        self.pricing_methodology = pricing_methodology
        self.data_directory = data_directory
        
        # Load data for both scenarios 
        self._load_phase_in_data()
        self._load_supporting_data()
        
        print(f"Initialized DCF Analysis V9 for {country_name}")
        print(f"Pricing methodology: {pricing_methodology}")
        print(f"Available technologies (2035): {list(self.phase_in_data_2035.keys())}")
        print(f"Available technologies (2050): {list(self.phase_in_data_2050.keys())}")

    def _load_phase_in_data(self):
        """Load phase-in data files for both scenarios"""
        self.phase_in_data_2035 = {}
        self.phase_in_data_2050 = {}
        
        scenarios = {
            '2035': self.phase_in_data_2035,
            'Net Zero 2050': self.phase_in_data_2050
        }
        
        for scenario_year, data_dict in scenarios.items():
            print(f"\nLoading {scenario_year} scenario data...")
            file_patterns = {
                'unit_investment_cost': f'battery_unit_ic_{self.country_code}.json',
                'yearly_available_capacity': f'battery_yearly_available_capacity_{scenario_year}_{self.country_code}.json',
                'yearly_installed_capacity': f'battery_yearly_installed_capacity_{scenario_year}_{self.country_code}.json'
            }
            
            for data_type, filename in file_patterns.items():
                filepath = Path(self.data_directory) / filename
                if filepath.exists():
                    print(f"  FOUND: {filename}")
                    try:
                        with open(filepath, 'r') as f:
                            file_data = json.load(f)
                        
                        for technology, tech_data in file_data.items():
                            if technology in ENERGY_MAPPING:
                                if technology not in data_dict:
                                    data_dict[technology] = {}
                                data_dict[technology][data_type] = tech_data
                    except Exception as e:
                        print(f"  ERROR loading {filename}: {e}")
                else:
                    print(f"  NOT FOUND: {filename}")
        
        print(f"\nData loading summary:")
        print(f"  2035 scenario technologies: {len(self.phase_in_data_2035)}")
        print(f"  2050 scenario technologies: {len(self.phase_in_data_2050)}")

    def _load_supporting_data(self):
        """Load capacity factors and revenue factors - FIXED FOR V12 CSV"""
        try:
            # Load capacity factors 
            self.capacity_factors = pd.read_csv('v3_capacity_weighted_average_capacity_factor.csv')
            
            # FIXED: Load revenue factors from V12 continental extrapolation
            self.revenue_factors = pd.read_csv('revenue_analysis_continental_extrapolation_v12.csv')
            
            # Get country-specific factors
            self.cap_factors = self.capacity_factors[
                self.capacity_factors['region'] == self.country_name
            ].iloc[0] if len(self.capacity_factors[self.capacity_factors['region'] == self.country_name]) > 0 else pd.Series()
            
            # FIXED: Get revenue factors using correct column names
            country_revenue_data = self.revenue_factors[
                self.revenue_factors['country_name'] == self.country_name  # FIXED: was 'Country'
            ]
            
            if len(country_revenue_data) > 0:
                # FIXED: Use correct column name from V12 extrapolation
                revenue_per_mwh = country_revenue_data.iloc[0]['weighted_avg_usd_mwh']  # FIXED: was 'revenue_per_mwh'
                self.rev_factors = pd.Series({
                    'Solar': revenue_per_mwh,
                    'Wind_Onshore': revenue_per_mwh,
                    'Wind_Offshore': revenue_per_mwh,
                    'Hydropower': revenue_per_mwh,
                    'Geothermal': revenue_per_mwh
                })
                print(f"Loaded revenue factors for {self.country_name}")
                print(f"Revenue per MWh: ${revenue_per_mwh:.2f}")
            else:
                print(f"No revenue data found for {self.country_name}, using defaults")
                self.rev_factors = pd.Series({
                    'Solar': 50.0,
                    'Wind_Onshore': 60.0,
                    'Wind_Offshore': 80.0,
                    'Hydropower': 70.0,
                    'Geothermal': 90.0
                })
                
        except Exception as e:
            print(f"Error loading supporting data: {e}")
            # Use defaults
            self.cap_factors = pd.Series({
                'Solar': 0.25,
                'Wind_Onshore': 0.35,
                'Wind_Offshore': 0.45,
                'Hydropower': 0.50,
                'Geothermal': 0.80
            })
            self.rev_factors = pd.Series({
                'Solar': 50.0,
                'Wind_Onshore': 60.0,
                'Wind_Offshore': 80.0,
                'Hydropower': 70.0,
                'Geothermal': 90.0
            })

    def extend_operational_capacity(self, available_capacity_data, energy_source, investment_end_year=2050):
        """Check data extent and return available capacity data"""
        years_with_data = [int(year) for year in available_capacity_data.keys() 
                          if available_capacity_data[year] > 0]
        
        if years_with_data:
            max_year = max(years_with_data)
            print(f"  {energy_source}: Data extends to {max_year}")
        
        return available_capacity_data

    def calculate_fund_scenario(self, investment_end_year=2050, integrator=None):
        """Calculate DCF for a single fund scenario WITH OPPORTUNITY COSTS"""
        yearly_investment_data = []
        yearly_revenue_data = []
        yearly_net_data = []
        technology_summary = {}
        
        # Select appropriate dataset based on investment end year
        if investment_end_year == 2035:
            phase_in_data = self.phase_in_data_2035
            scenario_name = "2035 scenario"
        else:
            phase_in_data = self.phase_in_data_2050
            scenario_name = "2050 scenario"
        
        print(f"\nUsing {scenario_name} dataset for {investment_end_year} fund")
        
        if not phase_in_data:
            print(f"No data available for {scenario_name}")
            return {
                'yearly_investment': pd.DataFrame(),
                'yearly_revenue': pd.DataFrame(),
                'yearly_net': pd.DataFrame(),
                'technology_summary': {},
                'investment_end_year': investment_end_year
            }
        
        # Process each renewable energy source
        for energy_source, csv_column in ENERGY_MAPPING.items():
            if energy_source not in phase_in_data:
                continue
            
            energy_data = phase_in_data[energy_source]
            
            # Get factors
            capacity_factor = self.cap_factors.get(csv_column, 0) if not self.cap_factors.empty else 0.3
            base_revenue_factor = self.rev_factors.get(csv_column, 0)
            wacc = DISCOUNT_RATE
            
            if capacity_factor <= 0 or base_revenue_factor <= 0:
                print(f"  {energy_source}: Skipped - capacity_factor: {capacity_factor}, revenue_factor: {base_revenue_factor}")
                continue
            
            available_capacity_data = energy_data.get('yearly_available_capacity', {})
            installed_capacity_data = energy_data.get('yearly_installed_capacity', {})
            unit_cost_data = energy_data.get('unit_investment_cost', {})
            
            if not available_capacity_data or not installed_capacity_data or not unit_cost_data:
                missing = []
                if not available_capacity_data: missing.append("available_capacity")
                if not installed_capacity_data: missing.append("installed_capacity")
                if not unit_cost_data: missing.append("unit_cost")
                print(f"  {energy_source}: Missing data - {missing}")
                continue
            
            extended_capacity_data = self.extend_operational_capacity(
                available_capacity_data, energy_source, investment_end_year
            )
            
            years_with_capacity = [
                int(year) for year in extended_capacity_data.keys()
                if extended_capacity_data[year] > 0
            ]
            
            if not years_with_capacity:
                continue
            
            max_revenue_year = max(years_with_capacity)
            print(f"  {energy_source}: Revenue calculated through {max_revenue_year}")
            
            # Calculate year-by-year cash flows
            for year in range(2025, max_revenue_year + 1):
                year_str = str(year)
                
                # Investment cash flow
                installed_capacity = installed_capacity_data.get(year_str, 0)  # kW
                unit_cost = unit_cost_data.get(year_str, 0)  # $/kW
                investment_amount = installed_capacity * unit_cost
                
                # Revenue cash flow
                available_capacity = extended_capacity_data.get(year_str, 0)  # kW
                annual_energy_production = (available_capacity * capacity_factor * 
                                          HOURS_IN_YEAR / 1000)  # MWh per year
                revenue_amount = annual_energy_production * base_revenue_factor
                
                # Net cash flow
                net_amount = revenue_amount - investment_amount
                
                # Store yearly data
                yearly_investment_data.append({
                    'year': year,
                    'technology': energy_source,
                    'investment': investment_amount,
                    'capacity_mw': installed_capacity
                })
                
                yearly_revenue_data.append({
                    'year': year,
                    'technology': energy_source,
                    'revenue': revenue_amount,
                    'energy_mwh': annual_energy_production
                })
                
                yearly_net_data.append({
                    'year': year,
                    'technology': energy_source,
                    'net_cash_flow': net_amount
                })
            
            # Technology summary
            technology_summary[energy_source] = {
                'total_capacity_mw': sum(installed_capacity_data.values()),
                'capacity_factor': capacity_factor,
                'revenue_factor_per_mwh': base_revenue_factor,
                'max_revenue_year': max_revenue_year
            }
        
        # Convert to DataFrames
        fund_results = {
            'yearly_investment': pd.DataFrame(yearly_investment_data),
            'yearly_revenue': pd.DataFrame(yearly_revenue_data),
            'yearly_net': pd.DataFrame(yearly_net_data),
            'technology_summary': technology_summary,
            'investment_end_year': investment_end_year
        }
        
        # OPPORTUNITY COST INTEGRATION (ONLY FOR YEARS 2025-2050)
        if integrator and not fund_results['yearly_net'].empty:
            # Get total annual cash flows
            yearly_totals = fund_results['yearly_net'].groupby('year')['net_cash_flow'].sum()
            years = sorted(yearly_totals.index)
            annual_cash_flows = [yearly_totals[year] for year in years]
            
            # Filter to only years where opportunity cost data exists (2025-2050)
            filtered_years = [year for year in years if 2025 <= year <= 2050]
            if filtered_years:
                filtered_cash_flows = [yearly_totals[year] for year in filtered_years]
                
                # Apply opportunity costs
                enhanced_cash_flows, opp_costs = add_opportunity_costs_to_dcf_cash_flows(
                    filtered_cash_flows, self.country_name, 2025, integrator
                )
                
                # Update net cash flows with opportunity costs (only for 2025-2050)
                for i, year in enumerate(filtered_years):
                    if i < len(enhanced_cash_flows):
                        mask = fund_results['yearly_net']['year'] == year
                        original_total = filtered_cash_flows[i]
                        enhancement = enhanced_cash_flows[i] - original_total
                        
                        # Proportionally enhance each technology's cash flow
                        if original_total != 0:
                            enhancement_factor = enhanced_cash_flows[i] / original_total
                            fund_results['yearly_net'].loc[mask, 'net_cash_flow'] *= enhancement_factor
                
                total_opp_value = sum(opp_costs)
                print(f"  Applied opportunity costs for {self.country_name}: ${total_opp_value:,.0f}")
        
        return fund_results

    def calculate_present_values(self, fund_results):
        """Calculate present values for investment, revenue, and net cash flows"""
        if fund_results['yearly_net'].empty:
            return 0, 0, 0
        
        investment_pv = 0
        revenue_pv = 0
        net_pv = 0
        
        # Investment PV
        if not fund_results['yearly_investment'].empty:
            for _, row in fund_results['yearly_investment'].iterrows():
                discount_factor = 1 / (1 + DISCOUNT_RATE) ** (row['year'] - 2025)
                investment_pv += row['investment'] * discount_factor
        
        # Revenue PV
        if not fund_results['yearly_revenue'].empty:
            for _, row in fund_results['yearly_revenue'].iterrows():
                discount_factor = 1 / (1 + DISCOUNT_RATE) ** (row['year'] - 2025)
                revenue_pv += row['revenue'] * discount_factor
        
        # Net PV
        net_pv = revenue_pv - investment_pv
        
        return investment_pv, revenue_pv, net_pv

    def create_timeline_plots(self, fund_a_results, fund_b_results):
        """Create timeline visualization with 4 focused graphs and HIGH CONTRAST COLORS"""
        fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(18, 14))
        fig.suptitle(f'{self.country_name} - {self.pricing_methodology.title()} Pricing\nRenewable Energy Investment Timeline Analysis', 
                    fontsize=16, fontweight='bold')
        
        # MUCH STRONGER CONTRAST COLORS
        fund_a_colors = {'investment': '#FF4444', 'revenue': '#FFD700', 'net': '#DC143C'}  # Red, Gold, Crimson
        fund_b_colors = {'investment': '#000080', 'revenue': '#008000', 'net': '#FF8C00'}  # Navy, Green, Dark Orange
        
        # Plot 1: Present Value of Investment Costs
        for fund_name, results, color in [
            ('Fund A (10-year commitment)', fund_a_results, fund_a_colors['investment']),
            ('Fund B (25-year commitment)', fund_b_results, fund_b_colors['investment'])
        ]:
            if not results['yearly_investment'].empty:
                yearly_data = results['yearly_investment'].groupby('year')['investment'].sum()
                yearly_pv = []
                for year, investment in yearly_data.items():
                    discount_factor = 1 / (1 + DISCOUNT_RATE) ** (year - 2025)
                    yearly_pv.append(investment * discount_factor / 1e9)
                
                ax1.bar(yearly_data.index, yearly_pv, alpha=0.8, label=fund_name, color=color, width=0.8)
        
        ax1.set_title('Present Value of Investment Costs', fontweight='bold', fontsize=14)
        ax1.set_xlabel('Year')
        ax1.set_ylabel('Investment PV ($ Billions)')
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        
        # Plot 2: Present Value of Revenue Streams
        for fund_name, results, color in [
            ('Fund A (10-year commitment)', fund_a_results, fund_a_colors['revenue']),
            ('Fund B (25-year commitment)', fund_b_results, fund_b_colors['revenue'])
        ]:
            if not results['yearly_revenue'].empty:
                yearly_data = results['yearly_revenue'].groupby('year')['revenue'].sum()
                yearly_pv = []
                for year, revenue in yearly_data.items():
                    discount_factor = 1 / (1 + DISCOUNT_RATE) ** (year - 2025)
                    yearly_pv.append(revenue * discount_factor / 1e9)
                
                ax2.bar(yearly_data.index, yearly_pv, alpha=0.8, label=fund_name, color=color, width=0.8)
        
        ax2.set_title('Present Value of Revenue Streams', fontweight='bold', fontsize=14)
        ax2.set_xlabel('Year')
        ax2.set_ylabel('Revenue PV ($ Billions)')
        ax2.legend()
        ax2.grid(True, alpha=0.3)
        
        # Plot 3: Annual Net Cash Flow (actual annual amounts)
        for fund_name, results, color in [
            ('Fund A (10-year commitment)', fund_a_results, fund_a_colors['net']),
            ('Fund B (25-year commitment)', fund_b_results, fund_b_colors['net'])
        ]:
            if not results['yearly_net'].empty:
                yearly_net_cf = results['yearly_net'].groupby('year')['net_cash_flow'].sum()
                ax3.bar(yearly_net_cf.index, yearly_net_cf.values / 1e9, alpha=0.8, 
                       label=fund_name, color=color, width=0.8)
        
        ax3.axhline(y=0, color='black', linestyle='--', alpha=0.7, linewidth=2)
        ax3.set_title('Annual Net Cash Flow by Year', fontweight='bold', fontsize=14)
        ax3.set_xlabel('Year')
        ax3.set_ylabel('Net Cash Flow ($ Billions)')
        ax3.legend()
        ax3.grid(True, alpha=0.3)
        
        # Plot 4: Project Breakeven Analysis with COLORED FILL AREAS
        for fund_name, results, color in [
            ('Fund A (10-year commitment)', fund_a_results, fund_a_colors['net']),
            ('Fund B (25-year commitment)', fund_b_results, fund_b_colors['net'])
        ]:
            if not results['yearly_net'].empty:
                yearly_net_cf = results['yearly_net'].groupby('year')['net_cash_flow'].sum()
                cumulative_cf = yearly_net_cf.cumsum() / 1e9
                
                # Plot the line
                line = ax4.plot(cumulative_cf.index, cumulative_cf.values, marker='o', 
                        label=fund_name, color=color, linewidth=4, markersize=8, alpha=0.9)
                
                # Add COLORED FILL AREAS (negative = red, positive = green)
                ax4.fill_between(cumulative_cf.index, cumulative_cf.values, 0, 
                               where=(cumulative_cf.values <= 0), color='red', alpha=0.2, 
                               interpolate=True, label='Negative Cash Flow' if fund_name == 'Fund A (10-year commitment)' else "")
                ax4.fill_between(cumulative_cf.index, cumulative_cf.values, 0, 
                               where=(cumulative_cf.values > 0), color='green', alpha=0.2, 
                               interpolate=True, label='Positive Cash Flow' if fund_name == 'Fund A (10-year commitment)' else "")
        
        # Clean breakeven line
        ax4.axhline(y=0, color='black', linestyle='-', alpha=0.8, linewidth=3)
        ax4.set_title('Project Breakeven Analysis\n(Cumulative Cash Flow)', fontweight='bold', fontsize=14)
        ax4.set_xlabel('Year')
        ax4.set_ylabel('Cumulative Cash Flow ($ Billions)')
        ax4.legend(loc='upper left')
        ax4.grid(True, alpha=0.4)
        
        # Simplified breakeven annotations
        for fund_name, results, color in [
            ('Fund A', fund_a_results, fund_a_colors['net']),
            ('Fund B', fund_b_results, fund_b_colors['net'])
        ]:
            if not results['yearly_net'].empty:
                yearly_net_cf = results['yearly_net'].groupby('year')['net_cash_flow'].sum()
                cumulative_cf = yearly_net_cf.cumsum()
                
                # Find breakeven year
                positive_years = cumulative_cf[cumulative_cf > 0]
                if len(positive_years) > 0:
                    breakeven_year = positive_years.index[0]
                    # Simple text annotation without arrows
                    ax4.text(breakeven_year, cumulative_cf[breakeven_year]/1e9 + abs(cumulative_cf.max()/1e9) * 0.05, 
                           f'{fund_name}\nBreakeven: {breakeven_year}', 
                           ha='center', va='bottom', fontsize=11, fontweight='bold', 
                           color=color, bbox=dict(boxstyle="round,pad=0.3", facecolor='white', alpha=0.8))
        
        plt.tight_layout()
        return fig

    def run_complete_dcf_analysis(self, integrator=None):
        """Execute complete DCF analysis WITH OPPORTUNITY COSTS"""
        print(f"\nRUNNING COMPLETE DCF ANALYSIS V9 - {self.country_name}")
        print("="*80)
        
        print("CALCULATING TWO FUND SCENARIOS")
        print("="*80)
        
        # Calculate both scenarios
        print("Calculating Fund A (10-year commitment)...")
        fund_a_results = self.calculate_fund_scenario(investment_end_year=2035, integrator=integrator)
        fund_a_inv_pv, fund_a_rev_pv, fund_a_net_npv = self.calculate_present_values(fund_a_results)
        
        print("Calculating Fund B (25-year commitment)...")
        fund_b_results = self.calculate_fund_scenario(investment_end_year=2050, integrator=integrator)
        fund_b_inv_pv, fund_b_rev_pv, fund_b_net_npv = self.calculate_present_values(fund_b_results)
        
        # Create timeline visualization
        timeline_fig = self.create_timeline_plots(fund_a_results, fund_b_results)
        
        # Print results 
        print(f"\n{'='*80}")
        print("FUND SCENARIOS COMPARISON")
        print(f"{'='*80}")
        print(f"{'Scenario':<35} {'Investment PV':<15} {'Revenue PV':<15} {'Net NPV':<15} {'NPV+':<5}")
        print("-" * 85)
        
        fund_a_npv_positive = "YES" if fund_a_net_npv > 0 else "NO"
        fund_b_npv_positive = "YES" if fund_b_net_npv > 0 else "NO"
        
        print(f"Fund A (10-year commitment)        ${fund_a_inv_pv/1e9:>13.1f}B ${fund_a_rev_pv/1e9:>13.1f}B ${fund_a_net_npv/1e9:>13.1f}B {fund_a_npv_positive:>4}")
        print(f"Fund B (25-year commitment)        ${fund_b_inv_pv/1e9:>13.1f}B ${fund_b_rev_pv/1e9:>13.1f}B ${fund_b_net_npv/1e9:>13.1f}B {fund_b_npv_positive:>4}")
        
        print(f"\nINVESTMENT ANALYSIS:")
        print(f"1. Fund A (10-year commitment) NPV Analysis:")
        print(f"   Result: {'POSITIVE' if fund_a_net_npv > 0 else 'NEGATIVE'} (NPV = ${fund_a_net_npv/1e9:.1f}B)")
        print(f"2. Fund B (25-year commitment) NPV Analysis:")
        print(f"   Result: {'POSITIVE' if fund_b_net_npv > 0 else 'NEGATIVE'} (NPV = ${fund_b_net_npv/1e9:.1f}B)")
        
        return {
            'country_name': self.country_name,
            'country_code': self.country_code,
            'fund_type': 'renewable_energy',
            'pricing_methodology': self.pricing_methodology,
            'initial_investment': fund_a_inv_pv,  # For Module B compatibility
            'annual_cash_flows': fund_a_results['yearly_net'].groupby('year')['net_cash_flow'].sum().tolist(),
            'cash_flow_years': sorted(fund_a_results['yearly_net']['year'].unique()),
            'project_irr': None,  # Will be calculated in Module B
            'fund_a': {
                'investment_pv': fund_a_inv_pv,
                'revenue_pv': fund_a_rev_pv,
                'net_npv': fund_a_net_npv,
                'npv_positive': fund_a_net_npv > 0,
                'results': fund_a_results
            },
            'fund_b': {
                'investment_pv': fund_b_inv_pv,
                'revenue_pv': fund_b_rev_pv,
                'net_npv': fund_b_net_npv,
                'npv_positive': fund_b_net_npv > 0,
                'results': fund_b_results
            },
            'timeline_figure': timeline_fig
        }

def run_multi_country_dcf_analysis():
    """
    Run DCF analysis for ALL countries with opportunity costs integration
    """
    print("="*80)
    print("MODULE A: DCF/NPV ANALYSIS V9 WITH OPPORTUNITY COSTS")
    print("154 Countries + Continental Extrapolation + PUB's Phase-out Costs")
    print("="*80)

    # Initialize opportunity cost integrator
    integrator = OpportunityCostIntegrator()

    # Load all countries from V12 continental extrapolation
    try:
        revenue_data = pd.read_csv('revenue_analysis_continental_extrapolation_v12.csv')
        countries = revenue_data['country_name'].unique()  # FIXED: correct column name
        print(f"Found {len(countries)} countries in revenue data")
        print(f"Including PUB's opportunity costs (2025-2050)")
    except Exception as e:
        print(f"Error loading revenue data: {e}")
        return None
    
    pricing_methodologies = ['weighted_avg']  # Simplified for efficiency
    
    # Results storage
    all_results = []
    detailed_results = []
    
    for country in countries:
        if country not in COMPLETE_COUNTRY_CODES:
            print(f"Skipping {country}: No country code mapping")
            continue
        
        country_code = COMPLETE_COUNTRY_CODES[country]
        
        for pricing_method in pricing_methodologies:
            try:
                print(f"\n{'='*60}")
                print(f"ANALYZING: {country} - {pricing_method.upper()}")
                print(f"{'='*60}")
                
                # Run DCF analysis with opportunity costs
                dcf = RenewableEnergyDCFV9(country, country_code, pricing_method)
                result = dcf.run_complete_dcf_analysis(integrator)
                
                # Store detailed results for Module B
                detailed_results.append(result)
                
                # Store summary results
                all_results.append({
                    'Country': country,
                    'Methodology': pricing_method,
                    'Fund_A_Investment_PV_B': result['fund_a']['investment_pv'] / 1e9,
                    'Fund_A_Revenue_PV_B': result['fund_a']['revenue_pv'] / 1e9,
                    'Fund_A_Net_NPV_B': result['fund_a']['net_npv'] / 1e9,
                    'Fund_A_NPV_Positive': result['fund_a']['npv_positive'],
                    'Fund_B_Investment_PV_B': result['fund_b']['investment_pv'] / 1e9,
                    'Fund_B_Revenue_PV_B': result['fund_b']['revenue_pv'] / 1e9,
                    'Fund_B_Net_NPV_B': result['fund_b']['net_npv'] / 1e9,
                    'Fund_B_NPV_Positive': result['fund_b']['npv_positive']
                })
                
                # Save visualization
                plt.figure(figsize=(18, 14))
                result['timeline_figure']
                plt.savefig(f"{country}_{pricing_method}_dcf_analysis.png", dpi=300, bbox_inches='tight')
                print(f"Saved: {country}_{pricing_method}_dcf_analysis.png")
                plt.show()
                
            except Exception as e:
                print(f"ERROR analyzing {country} - {pricing_method}: {e}")
    
    # Display summary
    if all_results:
        summary_df = pd.DataFrame(all_results)
        summary_df = summary_df.sort_values('Fund_B_Net_NPV_B', ascending=False)
        
        print(f"\n{'='*120}")
        print("COMPREHENSIVE SUMMARY - ALL COUNTRIES WITH OPPORTUNITY COSTS")
        print(f"{'='*120}")
        
        pd.set_option('display.max_columns', None)
        pd.set_option('display.width', None)
        pd.set_option('display.float_format', '{:.2f}'.format)
        
        print(summary_df.to_string(index=False))
        
        # Summary statistics
        positive_npv_count = len(summary_df[summary_df['Fund_B_Net_NPV_B'] > 0])
        total_analyses = len(summary_df)
        
        print(f"\n{'='*60}")
        print("SUMMARY STATISTICS")
        print(f"{'='*60}")
        print(f"Countries analyzed: {len(set(summary_df['Country']))}")
        print(f"Total analyses completed: {total_analyses}")
        print(f"Positive NPV projects (Fund B): {positive_npv_count} ({100*positive_npv_count/total_analyses:.1f}%)")
        print(f"Average Fund B NPV: ${summary_df['Fund_B_Net_NPV_B'].mean():.2f}B")
        print(f"Total portfolio NPV (Fund B): ${summary_df['Fund_B_Net_NPV_B'].sum():.2f}B")
        
        # Top 5 performers
        print(f"\nTop 5 Countries by Fund B NPV (with opportunity costs):")
        top_5 = summary_df.nlargest(5, 'Fund_B_Net_NPV_B')
        for _, row in top_5.iterrows():
            print(f"  {row['Country']}: ${row['Fund_B_Net_NPV_B']:.2f}B")
    
    print(f"\nModule A V9 Complete with Opportunity Cost Integration!")
    print("Ready for Module B (IRR Analysis)")
    
    return detailed_results

# Execute the analysis
if __name__ == "__main__":
    module_a_results = run_multi_country_dcf_analysis()
    
    if module_a_results:
        print(f"\nSUCCESS: Module A returned {len(module_a_results)} detailed results")
        print("Ready for Module B integration!")
    else:
        print("ERROR: Module A returned None")
    
    plt.show()

In [ ]:
module_a_results = run_multi_country_dcf_analysis()

In [ ]:
"""
Corrected Module B - Clean IRR Only
"""
class ProjectIRRCalculator:
      """
      Module B: Calculate project-level IRRs 
      Fixed to match actual Module A output structure
      """

      def __init__(self, module_a_results: list):
          self.module_a_results = module_a_results
          self.irr_results = []

          # Country code mapping (since it's not in the results)
          self.country_codes = {
              'Argentina': 'AR', 'Brazil': 'BR', 'Chile': 'CL', 'India': 'IN', 'Indonesia': 'ID',
              'Bangladesh': 'BD', 'Philippines': 'PH', 'Thailand': 'TH', 'Malaysia': 'MY',
              'South Africa': 'ZA', 'Egypt': 'EG', 'Morocco': 'MA', 'Kenya': 'KE',
              'Kazakhstan': 'KZ', 'Pakistan': 'PK', 'Peru': 'PE', 'Colombia': 'CO',
              'China': 'CN', 'Mexico': 'MX', 'Turkey': 'TR', 'UAE': 'AE',
              'Myanmar': 'MM', 'Nigeria': 'NG', 'Senegal': 'SN', 'Sri Lanka': 'LK',
              'Taiwan': 'TW', 'Zambia': 'ZM', 'Saudi Arabia': 'SA'
          }

      def extract_cash_flows(self, fund_results: dict) -> tuple:
          """Extract annual cash flows from fund results"""
          yearly_net = fund_results['yearly_net']
          yearly_investment = fund_results['yearly_investment']

          if yearly_net.empty:
              return 0, [], []

          # Total initial investment
          total_investment = yearly_investment['investment'].sum() if not yearly_investment.empty else 0

          # Annual net cash flows by year
          annual_cf = yearly_net.groupby('year')['net_cash_flow'].sum().sort_index()

          return total_investment, annual_cf.values.tolist(), annual_cf.index.tolist()

      def calculate_project_irr(self, initial_investment: float, annual_cash_flows: list) -> tuple:
          """Calculate IRR using numpy-financial"""
          if initial_investment <= 0 or not annual_cash_flows:
              return None, "INSUFFICIENT_DATA"

          # Construct cash flow array: [-initial_investment, cf1, cf2, ..., cfN]
          full_cash_flows = [-initial_investment] + annual_cash_flows

          try:
              irr = npf.irr(full_cash_flows)
              if np.isnan(irr) or np.isinf(irr):
                  return None, "NO_SOLUTION"
              return float(irr), "SUCCESS"
          except:
              return None, "CALCULATION_ERROR"

      def process_all_scenarios(self) -> list:
          """Process ALL countries - fixed to match actual data structure"""
          for result in self.module_a_results:
              # Extract data using actual key names
              country_name = result['country_name']  # Note: 'country' not 'country_name'
              pricing_method = result['pricing_methodology']
              country_code = self.country_codes.get(country_name, 'XX')  # Get code from mapping

              # Fund A IRR
              fund_a_investment, fund_a_flows, fund_a_years = self.extract_cash_flows(result['fund_a']['results'])
              fund_a_irr, fund_a_status = self.calculate_project_irr(fund_a_investment, fund_a_flows)

              # Fund B IRR  
              fund_b_investment, fund_b_flows, fund_b_years = self.extract_cash_flows(result['fund_b']['results'])
              fund_b_irr, fund_b_status = self.calculate_project_irr(fund_b_investment, fund_b_flows)

              # Store both results
              self.irr_results.extend([
                  {
                      'country_name': country_name,
                      'country_code': country_code,
                      'pricing_methodology': pricing_method,
                      'fund_type': 'Fund A',
                      'fund_description': '10-year commitment (2025-2035)',
                      'project_irr': fund_a_irr,
                      'irr_status': fund_a_status,
                      'initial_investment': fund_a_investment,
                      'annual_cash_flows': fund_a_flows,
                      'cash_flow_years': fund_a_years,
                      'project_npv': result['fund_a']['net_npv'],
                      'years_of_cashflow': len(fund_a_flows)
                  },
                  {
                      'country_name': country_name,
                      'country_code': country_code,
                      'pricing_methodology': pricing_method,
                      'fund_type': 'Fund B',
                      'fund_description': '25-year commitment (2025-2050)',
                      'project_irr': fund_b_irr,
                      'irr_status': fund_b_status,
                      'initial_investment': fund_b_investment,
                      'annual_cash_flows': fund_b_flows,
                      'cash_flow_years': fund_b_years,
                      'project_npv': result['fund_b']['net_npv'],
                      'years_of_cashflow': len(fund_b_flows)
                  }
              ])

          return self.irr_results

      def create_complete_results_table(self) -> pd.DataFrame:
          """Create COMPLETE results table showing ALL countries/funds/methodologies"""
          df = pd.DataFrame(self.irr_results)

          # Format IRR as percentage
          df['IRR_Percent'] = df['project_irr'].apply(
              lambda x: f"{x*100:.2f}%" if x is not None else "N/A"
          )

          # Format monetary values in billions
          df['Investment_B'] = (df['initial_investment'] / 1e9).round(2)
          df['NPV_B'] = (df['project_npv'] / 1e9).round(2)

          # Select display columns
          display_df = df[[
              'country_name', 'country_code', 'fund_type', 'pricing_methodology',
              'IRR_Percent', 'Investment_B', 'NPV_B', 'years_of_cashflow', 'irr_status'
          ]].copy()

          # Sort for readability: country, then fund, then methodology
          display_df = display_df.sort_values(['country_name', 'fund_type', 'pricing_methodology'])

          return display_df

def run_module_b_irr_analysis(module_a_results):
      """Execute Module B - Show ALL results, no WACC"""
      print("="*80)
      print("MODULE B: PROJECT-LEVEL IRR ANALYSIS")
      print("Calculates IRR for all Fund/Country/Pricing combinations")
      print("="*80)

      irr_calculator = ProjectIRRCalculator(module_a_results)
      irr_results = irr_calculator.process_all_scenarios()

      # Display COMPLETE results table
      complete_table = irr_calculator.create_complete_results_table()

      print(f"\nCOMPLETE IRR RESULTS TABLE ({len(complete_table)} scenarios):")
      print("="*120)

      # Show ALL rows
      pd.set_option('display.max_rows', None)
      pd.set_option('display.max_columns', None)
      pd.set_option('display.width', None)

      print(complete_table.to_string(index=False))

      # Summary statistics
      successful = complete_table[complete_table['irr_status'] == 'SUCCESS']
      failed = complete_table[complete_table['irr_status'] != 'SUCCESS']

      print(f"\n{'='*80}")
      print("MODULE B SUMMARY STATISTICS:")
      print(f"{'='*80}")
      print(f"Total scenarios processed: {len(complete_table)}")
      print(f"Successful IRR calculations: {len(successful)} ({len(successful)/len(complete_table)*100:.1f}%)")
      print(f"Failed calculations: {len(failed)} ({len(failed)/len(complete_table)*100:.1f}%)")

      if len(successful) > 0:
          # Parse IRR values for statistics
          irr_values = []
          for irr_str in successful['IRR_Percent']:
              if irr_str != "N/A":
                  irr_values.append(float(irr_str.replace('%', '')) / 100)

          if irr_values:
              print(f"IRR Statistics for successful calculations:")
              print(f"  Mean IRR: {np.mean(irr_values)*100:.2f}%")
              print(f"  Median IRR: {np.median(irr_values)*100:.2f}%")
              print(f"  IRR Range: {min(irr_values)*100:.2f}% to {max(irr_values)*100:.2f}%")

              # Show failure breakdown if any
              if len(failed) > 0:
                  print(f"\nFailure breakdown:")
                  failure_counts = failed['irr_status'].value_counts()
                  for status, count in failure_counts.items():
                      print(f"  {status}: {count} scenarios")

      print(f"\nModule B Complete! Ready for Module C (Waterfall Analysis)")
      print(f"Output: {len(irr_results)} detailed IRR results")

      return irr_results

In [ ]:
module_b_results = run_module_b_irr_analysis(module_a_results)

In [ ]:
"""
MODULE C - WATERFALL ANALYSIS V1
Tranche-level cash flow distribution and IRR analysis
Implements both Version 1 (year-by-year) and Version 2 (retrospective with␣
↪interest)
"""
class WaterfallAnalyzer:
"""
Module C: Waterfall Analysis for Renewable Energy Project Finance
Capital Structure:
- Senior Debt: 50%
- Sub Debt: 25%
- Junior Equity: 25%
Two Versions:
- Version 1: Year-by-year (unpaid amounts written off)
- Version 2: Retrospective (unpaid amounts compound with interest)
"""
def __init__(self, module_b_results: list, aug22_csv_path: str = 'Aug22.
↪csv'):
self.module_b_results = module_b_results
self.aug22_csv_path = aug22_csv_path
self.waterfall_results = []
# Capital structure percentages
self.capital_structure = {
'Senior_Debt': 0.50,
'Sub_Debt': 0.25,
'Equity': 0.25
}
# Load required returns data
self.required_returns = self._load_required_returns()
print(f"Initialized Waterfall Analyzer for {len(module_b_results)}␣
↪scenarios")
print(f"Capital Structure: Senior Debt (50%), Sub Debt (25%), Equity␣
↪(25%)")
def _load_required_returns(self) -> pd.DataFrame:
"""Load and parse required returns from Aug22.csv"""
try:
df = pd.read_csv(self.aug22_csv_path)
print(f"Loaded required returns for {len(df)} countries from {self.
↪aug22_csv_path}")
return df
except Exception as e:
print(f"Error loading required returns: {e}")
return pd.DataFrame()
def _parse_percentage_safely(self, value) -> float:
"""Safely parse percentage values from CSV"""
if pd.isna(value) or value == '':
return 0.0
try:
if isinstance(value, str):
    # Remove % sign and convert to decimal
clean_value = value.replace('%'
,
return float(clean_value) / 100
'').strip()
else:
# Already a number, assume it's in decimal form
return float(value)
except:
return 0.0
def _get_required_returns(self, country_name: str) -> Dict[str, float]:
"""Get required returns for a specific country"""
if self.required_returns.empty:
# Fallback default rates if CSV not available
return {
'Senior_Debt': 0.06, # 6%
'Sub_Debt': 0.08, # 8%
'Equity': 0.12 # 12%
}
# Find country in required returns data
country_row = self.required_returns[
self.required_returns['Country'].str.contains(country_name,␣
↪case=False, na=False)
]
if len(country_row) == 0:
print(f"Warning: No required returns found for {country_name},␣
↪using defaults")
return {
'Senior_Debt': 0.06,
'Sub_Debt': 0.08,
'Equity': 0.12
}
row = country_row.iloc[0]
return {
'Senior_Debt': self._parse_percentage_safely(row.
↪get('new_woinf_sen_debt')),
'Sub_Debt': self._parse_percentage_safely(row.
↪get('new_woinf_sub_debt')),
'Equity': self._parse_percentage_safely(row.get('new_woinf_equity'))
}
def _calculate_tranche_investments(self, total_investment: float) ->␣
↪Dict[str, float]:
"""Calculate investment allocation by tranche"""
return {
    'Senior_Debt': total_investment * self.
↪capital_structure['Senior_Debt'],
'Sub_Debt': total_investment * self.capital_structure['Sub_Debt'],
'Equity': total_investment * self.capital_structure['Equity']
}
def _calculate_annual_requirements(self, tranche_investments: Dict[str,␣
↪float],
required_returns: Dict[str, float]) ->␣
↪Dict[str, float]:
return {
"""Calculate annual payment requirements (interest-only approach)"""
tranche: investment * required_returns[tranche]
for tranche, investment in tranche_investments.items()
}
def _version_1_waterfall(self, annual_cash_flows: List[float],
annual_requirements: Dict[str, float]) -> Dict[str,␣
↪List[float]]:
"""
Version 1: Year-by-year waterfall (simple)
Unpaid amounts are written off each year
"""
payments = {tranche: [] for tranche in self.capital_structure.keys()}
for yearly_cash in annual_cash_flows:
available_cash = yearly_cash
# Priority 1: Senior Debt
senior_payment = min(available_cash,␣
↪annual_requirements['Senior_Debt'])
payments['Senior_Debt'].append(max(0, senior_payment))
available_cash -= senior_payment
# Priority 2: Sub Debt
sub_payment = min(max(0, available_cash),␣
↪annual_requirements['Sub_Debt'])
payments['Sub_Debt'].append(max(0, sub_payment))
available_cash -= sub_payment
# Priority 3: Equity (gets remainder)
equity_payment = max(0, available_cash)
payments['Equity'].append(equity_payment)
return payments
def _version_2_waterfall(self, annual_cash_flows: List[float],
annual_requirements: Dict[str, float],
required_returns: Dict[str, float]) -> Dict[str,␣
↪List[float]]:
"""
Version 2: Retrospective with interest compounding
Unpaid amounts carry forward and compound at tranche rate
"""
payments = {tranche: [] for tranche in self.capital_structure.keys()}
# Track cumulative unpaid amounts
unpaid_amounts = {
'Senior_Debt': 0.0,
'Sub_Debt': 0.0,
'Equity': 0.0
}
for yearly_cash in annual_cash_flows:
available_cash = yearly_cash
# Compound unpaid amounts with interest
for tranche in unpaid_amounts:
unpaid_amounts[tranche] *= (1 + required_returns[tranche])
# Add current year requirements to unpaid amounts
total_owed = {
tranche: unpaid_amounts[tranche] + annual_requirements[tranche]
for tranche in self.capital_structure.keys()
}
# Priority 1: Senior Debt (including backlog)
senior_payment = min(available_cash, total_owed['Senior_Debt'])
payments['Senior_Debt'].append(max(0, senior_payment))
unpaid_amounts['Senior_Debt'] = max(0, total_owed['Senior_Debt'] -
␣
↪senior_payment)
available_cash -= senior_payment
# Priority 2: Sub Debt (including backlog)
sub_payment = min(max(0, available_cash), total_owed['Sub_Debt'])
payments['Sub_Debt'].append(max(0, sub_payment))
unpaid_amounts['Sub_Debt'] = max(0, total_owed['Sub_Debt'] -
␣
↪sub_payment)
available_cash -= sub_payment
# Priority 3: Equity (including backlog)
equity_payment = min(max(0, available_cash), total_owed['Equity'])
payments['Equity'].append(equity_payment)
unpaid_amounts['Equity'] = max(0, total_owed['Equity'] -
␣
↪equity_payment)
return payments
def _calculate_tranche_irr(self, tranche_investment: float,
annual_payments: List[float]) -> Tuple[float, str]:
"""Calculate IRR for a specific tranche"""
if tranche_investment <= 0 or not annual_payments:
return None,
"INSUFFICIENT_DATA"
# Cash flows: [-investment, payment1, payment2, ..., paymentN]
cash_flows = [-tranche_investment] + annual_payments
try:
irr = npf.irr(cash_flows)
if np.isnan(irr) or np.isinf(irr):
return None,
"NO_SOLUTION"
return float(irr), "SUCCESS"
except:
return None,
"CALCULATION_ERROR"
def process_all_scenarios(self) -> List[dict]:
"""Process waterfall analysis for all scenarios"""
for scenario in self.module_b_results:
country_name = scenario['country_name']
country_code = scenario['country_code']
fund_type = scenario['fund_type']
pricing_method = scenario['pricing_methodology']
total_investment = scenario['initial_investment']
annual_cash_flows = scenario['annual_cash_flows']
project_irr = scenario['project_irr']
# Get required returns for this country
required_returns = self._get_required_returns(country_name)
# Calculate tranche investments
tranche_investments = self.
↪_calculate_tranche_investments(total_investment)
# Calculate annual requirements
annual_requirements = self._calculate_annual_requirements(
tranche_investments, required_returns
)
# Process both versions
for version_name, waterfall_func in [
    ("Version_1"
("Version_2"
, self._version_1_waterfall),
, self._version_2_waterfall)
]:
# Apply waterfall logic
if version_name == "Version_1":
tranche_payments = waterfall_func(annual_cash_flows,␣
↪annual_requirements)
else:
tranche_payments = waterfall_func(annual_cash_flows,␣
↪annual_requirements, required_returns)
# Calculate tranche IRRs
for tranche_name in self.capital_structure.keys():
tranche_irr, irr_status = self._calculate_tranche_irr(
tranche_investments[tranche_name],
tranche_payments[tranche_name]
)
# Calculate performance metrics
required_return = required_returns[tranche_name]
excess_return = (tranche_irr - required_return) if␣
↪tranche_irr is not None else None
total_payments = sum(tranche_payments[tranche_name])
# Store result
self.waterfall_results.append({
'country_name': country_name,
'country_code': country_code,
'fund_type': fund_type,
'pricing_methodology': pricing_method,
'waterfall_version': version_name,
'tranche': tranche_name,
'tranche_investment': tranche_investments[tranche_name],
'tranche_irr': tranche_irr,
'required_return': required_return,
'excess_return': excess_return,
'irr_status': irr_status,
'total_payments_received': total_payments,
'annual_payments': tranche_payments[tranche_name],
'payment_years': scenario['cash_flow_years'],
'project_irr': project_irr,
'attractive_investment': (tranche_irr >␣
↪required_return) if tranche_irr is not None else False
})
return self.waterfall_results
def create_summary_table(self) -> pd.DataFrame:
"""Create comprehensive summary table"""
df = pd.DataFrame(self.waterfall_results)
# Format key columns
df['Tranche_IRR_Pct'] = df['tranche_irr'].apply(
lambda x: f"{x*100:.2f}%" if x is not None else "N/A"
)
df['Required_Return_Pct'] = df['required_return'].apply(lambda x:␣
↪f"{x*100:.2f}%")
df['Excess_Return_Pct'] = df['excess_return'].apply(
lambda x: f"{x*100:.2f}%" if x is not None else "N/A"
)
df['Investment_M'] = (df['tranche_investment'] / 1e6).round(1)
df['Total_Payments_M'] = (df['total_payments_received'] / 1e6).round(1)
↪
↪
↪
# Select display columns
display_cols = [
'country_name'
,
'fund_type'
,
'waterfall_version'
,
'tranche'
,
'Tranche_IRR_Pct'
,
'Excess_Return_Pct'
,
'Investment_M'
,
'Total_Payments_M'
'pricing_methodology'
,␣
'Required_Return_Pct'
,␣
,
'attractive_investment'
,␣
'irr_status'
]
summary_df = df[display_cols].copy()
summary_df = summary_df.sort_values([
'country_name'
,
'fund_type'
,
'waterfall_version'
'tranche'
,
'pricing_methodology'
,
])
return summary_df
def create_waterfall_visualizations(self) -> plt.Figure:
"""Create comprehensive waterfall analysis visualizations"""
df = pd.DataFrame(self.waterfall_results)
if df.empty:
print("No data available for visualization")
return None
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(20, 16))
fig.suptitle('Waterfall Analysis
             # Colors for tranches
tranche_colors = {
'Senior_Debt': '#1f77b4'
, # Blue
'Sub_Debt': '#ff7f0e'
, # Orange
'Equity': '#2ca02c' # Green
}
↪tranche]
# Plot 1: IRR vs Required Return by Tranche
successful_df = df[df['tranche_irr'].notna()].copy()
if not successful_df.empty:
for tranche in tranche_colors.keys():
tranche_data = successful_df[successful_df['tranche'] ==␣
if not tranche_data.empty:
x_pos = np.arange(len(tranche_data))
ax1.scatter(tranche_data['required_return'] * 100,
tranche_data['tranche_irr'] * 100,
c=tranche_colors[tranche], label=tranche, alpha=0.
↪7, s=50)
# Add diagonal line (IRR = Required Return)
max_val = max(successful_df['required_return'].max() * 100,
successful_df['tranche_irr'].max() * 100) if not␣
↪successful_df.empty else 15
ax1.plot([0, max_val], [0, max_val], 'k--', alpha=0.5,␣
↪label='Break-even')
ax1.set_xlabel('Required Return (%)')
ax1.set_ylabel('Tranche IRR (%)')
ax1.set_title('Tranche IRR vs Required Return', fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)
# Plot 2: Version Comparison (Version 2 - Version 1)
version_comparison = []
for country in df['country_name'].unique():
for fund in df['fund_type'].unique():
for pricing in df['pricing_methodology'].unique():
for tranche in tranche_colors.keys():
v1_data = df[(df['country_name'] == country) &
(df['fund_type'] == fund) &
(df['pricing_methodology'] == pricing) &
(df['tranche'] == tranche) &
(df['waterfall_version'] == 'Version_1')]
v2_data = df[(df['country_name'] == country) &
(df['fund_type'] == fund) &
(df['pricing_methodology'] == pricing) &
(df['tranche'] == tranche) &

In [ ]:
"""
MODULE C - WATERFALL ANALYSIS V1
Tranche-level cash flow distribution and IRR analysis
Implements both Version 1 (year-by-year) and Version 2 (retrospective with interest)
PRIORITY: ADD PLOT DISPLAYING HURDLE RATE COMPARISON WITH TRANCHE RETURNS 
for each country's tranche
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy_financial as npf
from typing import Dict, List, Tuple

class WaterfallAnalyzer:
    """
    Module C: Waterfall Analysis for Renewable Energy Project Finance
    Capital Structure:
    - Senior Debt: 50%
    - Sub Debt: 25%
    - Junior Equity: 25%
    
    Two Versions:
    - Version 1: Year-by-year (unpaid amounts written off)
    - Version 2: Retrospective (unpaid amounts compound with interest)
    """
    
    def __init__(self, module_b_results: list, aug22_csv_path: str = 'Aug22.csv'):
        self.module_b_results = module_b_results
        self.aug22_csv_path = aug22_csv_path
        self.waterfall_results = []
        
        # Capital structure percentages
        self.capital_structure = {
            'Senior_Debt': 0.50,
            'Sub_Debt': 0.25,
            'Equity': 0.25
        }
        
        # Load required returns data
        self.required_returns = self._load_required_returns()
        print(f"Initialized Waterfall Analyzer for {len(module_b_results)} scenarios")
        print(f"Capital Structure: Senior Debt (50%), Sub Debt (25%), Equity (25%)")

    def _load_required_returns(self) -> pd.DataFrame:
        """Load and parse required returns from Aug22.csv"""
        try:
            df = pd.read_csv(self.aug22_csv_path)
            print(f"Loaded required returns for {len(df)} countries from {self.aug22_csv_path}")
            return df
        except Exception as e:
            print(f"Error loading required returns: {e}")
            return pd.DataFrame()

    def _parse_percentage_safely(self, value) -> float:
        """Safely parse percentage values from CSV"""
        if pd.isna(value) or value == '':
            return 0.0
        try:
            if isinstance(value, str):
                # Remove % sign and convert to decimal
                clean_value = value.replace('%', '').strip()
                return float(clean_value) / 100
            else:
                # Already a number, assume it's in decimal form
                return float(value)
        except:
            return 0.0

    def _get_required_returns(self, country_name: str) -> Dict[str, float]:
        """Get required returns for a specific country"""
        if self.required_returns.empty:
            # Fallback default rates if CSV not available
            return {
                'Senior_Debt': 0.06,  # 6%
                'Sub_Debt': 0.08,    # 8%
                'Equity': 0.12       # 12%
            }
        
        # Find country in required returns data
        country_row = self.required_returns[
            self.required_returns['Country'].str.contains(country_name, case=False, na=False)
        ]
        
        if len(country_row) == 0:
            print(f"Warning: No required returns found for {country_name}, using defaults")
            return {
                'Senior_Debt': 0.06,
                'Sub_Debt': 0.08,
                'Equity': 0.12
            }
        
        row = country_row.iloc[0]
        return {
            'Senior_Debt': self._parse_percentage_safely(row.get('new_woinf_sen_debt')),
            'Sub_Debt': self._parse_percentage_safely(row.get('new_woinf_sub_debt')),
            'Equity': self._parse_percentage_safely(row.get('new_woinf_equity'))
        }

    def _calculate_tranche_investments(self, total_investment: float) -> Dict[str, float]:
        """Calculate investment allocation by tranche"""
        return {
            'Senior_Debt': total_investment * self.capital_structure['Senior_Debt'],
            'Sub_Debt': total_investment * self.capital_structure['Sub_Debt'],
            'Equity': total_investment * self.capital_structure['Equity']
        }

    def _calculate_annual_requirements(self, tranche_investments: Dict[str, float],
                                     required_returns: Dict[str, float]) -> Dict[str, float]:
        """Calculate annual payment requirements (interest-only approach)"""
        return {
            tranche: investment * required_returns[tranche]
            for tranche, investment in tranche_investments.items()
        }

    def _version_1_waterfall(self, annual_cash_flows: List[float],
                           annual_requirements: Dict[str, float]) -> Dict[str, List[float]]:
        """
        Version 1: Year-by-year waterfall (simple)
        Unpaid amounts are written off each year
        """
        payments = {tranche: [] for tranche in self.capital_structure.keys()}
        
        for yearly_cash in annual_cash_flows:
            available_cash = yearly_cash
            
            # Priority 1: Senior Debt
            senior_payment = min(available_cash, annual_requirements['Senior_Debt'])
            payments['Senior_Debt'].append(max(0, senior_payment))
            available_cash -= senior_payment
            
            # Priority 2: Sub Debt
            sub_payment = min(max(0, available_cash), annual_requirements['Sub_Debt'])
            payments['Sub_Debt'].append(max(0, sub_payment))
            available_cash -= sub_payment
            
            # Priority 3: Equity (gets remainder)
            equity_payment = max(0, available_cash)
            payments['Equity'].append(equity_payment)
        
        return payments

    def _version_2_waterfall(self, annual_cash_flows: List[float],
                           annual_requirements: Dict[str, float],
                           required_returns: Dict[str, float]) -> Dict[str, List[float]]:
        """
        Version 2: Retrospective with interest compounding
        Unpaid amounts carry forward and compound at tranche rate
        """
        payments = {tranche: [] for tranche in self.capital_structure.keys()}
        
        # Track cumulative unpaid amounts
        unpaid_amounts = {
            'Senior_Debt': 0.0,
            'Sub_Debt': 0.0,
            'Equity': 0.0
        }
        
        for yearly_cash in annual_cash_flows:
            available_cash = yearly_cash
            
            # Compound unpaid amounts with interest
            for tranche in unpaid_amounts:
                unpaid_amounts[tranche] *= (1 + required_returns[tranche])
            
            # Add current year requirements to unpaid amounts
            total_owed = {
                tranche: unpaid_amounts[tranche] + annual_requirements[tranche]
                for tranche in self.capital_structure.keys()
            }
            
            # Priority 1: Senior Debt (including backlog)
            senior_payment = min(available_cash, total_owed['Senior_Debt'])
            payments['Senior_Debt'].append(max(0, senior_payment))
            unpaid_amounts['Senior_Debt'] = max(0, total_owed['Senior_Debt'] - senior_payment)
            available_cash -= senior_payment
            
            # Priority 2: Sub Debt (including backlog)
            sub_payment = min(max(0, available_cash), total_owed['Sub_Debt'])
            payments['Sub_Debt'].append(max(0, sub_payment))
            unpaid_amounts['Sub_Debt'] = max(0, total_owed['Sub_Debt'] - sub_payment)
            available_cash -= sub_payment
            
            # Priority 3: Equity (including backlog)
            equity_payment = min(max(0, available_cash), total_owed['Equity'])
            payments['Equity'].append(equity_payment)
            unpaid_amounts['Equity'] = max(0, total_owed['Equity'] - equity_payment)
        
        return payments

    def _calculate_tranche_irr(self, tranche_investment: float,
                             annual_payments: List[float]) -> Tuple[float, str]:
        """Calculate IRR for a specific tranche"""
        if tranche_investment <= 0 or not annual_payments:
            return None, "INSUFFICIENT_DATA"
        
        # Cash flows: [-investment, payment1, payment2, ..., paymentN]
        cash_flows = [-tranche_investment] + annual_payments
        
        try:
            irr = npf.irr(cash_flows)
            if np.isnan(irr) or np.isinf(irr):
                return None, "NO_SOLUTION"
            return float(irr), "SUCCESS"
        except:
            return None, "CALCULATION_ERROR"

    def process_all_scenarios(self) -> List[dict]:
        """Process waterfall analysis for all scenarios"""
        for scenario in self.module_b_results:
            country_name = scenario['country_name']
            country_code = scenario['country_code']
            fund_type = scenario['fund_type']
            pricing_method = scenario['pricing_methodology']
            total_investment = scenario['initial_investment']
            annual_cash_flows = scenario['annual_cash_flows']
            project_irr = scenario['project_irr']
            
            # Get required returns for this country
            required_returns = self._get_required_returns(country_name)
            
            # Calculate tranche investments
            tranche_investments = self._calculate_tranche_investments(total_investment)
            
            # Calculate annual requirements
            annual_requirements = self._calculate_annual_requirements(
                tranche_investments, required_returns
            )
            
            # Process both versions
            for version_name, waterfall_func in [
                ("Version_1", self._version_1_waterfall),
                ("Version_2", self._version_2_waterfall)
            ]:
                # Apply waterfall logic
                if version_name == "Version_1":
                    tranche_payments = waterfall_func(annual_cash_flows, annual_requirements)
                else:
                    tranche_payments = waterfall_func(annual_cash_flows, annual_requirements, required_returns)
                
                # Calculate tranche IRRs
                for tranche_name in self.capital_structure.keys():
                    tranche_irr, irr_status = self._calculate_tranche_irr(
                        tranche_investments[tranche_name],
                        tranche_payments[tranche_name]
                    )
                    
                    # Calculate performance metrics
                    required_return = required_returns[tranche_name]
                    excess_return = (tranche_irr - required_return) if tranche_irr is not None else None
                    total_payments = sum(tranche_payments[tranche_name])
                    
                    # Store result
                    self.waterfall_results.append({
                        'country_name': country_name,
                        'country_code': country_code,
                        'fund_type': fund_type,
                        'pricing_methodology': pricing_method,
                        'waterfall_version': version_name,
                        'tranche': tranche_name,
                        'tranche_investment': tranche_investments[tranche_name],
                        'tranche_irr': tranche_irr,
                        'required_return': required_return,
                        'excess_return': excess_return,
                        'irr_status': irr_status,
                        'total_payments_received': total_payments,
                        'annual_payments': tranche_payments[tranche_name],
                        'payment_years': scenario['cash_flow_years'],
                        'project_irr': project_irr,
                        'attractive_investment': (tranche_irr > required_return) if tranche_irr is not None else False
                    })
        
        return self.waterfall_results

    def create_summary_table(self) -> pd.DataFrame:
        """Create comprehensive summary table"""
        df = pd.DataFrame(self.waterfall_results)
        
        # Format key columns
        df['Tranche_IRR_Pct'] = df['tranche_irr'].apply(
            lambda x: f"{x*100:.2f}%" if x is not None else "N/A"
        )
        df['Required_Return_Pct'] = df['required_return'].apply(lambda x: f"{x*100:.2f}%")
        df['Excess_Return_Pct'] = df['excess_return'].apply(
            lambda x: f"{x*100:.2f}%" if x is not None else "N/A"
        )
        df['Investment_M'] = (df['tranche_investment'] / 1e6).round(1)
        df['Total_Payments_M'] = (df['total_payments_received'] / 1e6).round(1)
        
        # Select display columns
        display_cols = [
            'country_name', 'pricing_methodology', 'fund_type', 'waterfall_version',
            'tranche', 'Tranche_IRR_Pct', 'Required_Return_Pct', 'Excess_Return_Pct',
            'Investment_M', 'Total_Payments_M', 'attractive_investment', 'irr_status'
        ]
        
        summary_df = df[display_cols].copy()
        summary_df = summary_df.sort_values([
            'country_name', 'fund_type', 'waterfall_version', 'pricing_methodology', 'tranche'
        ])
        
        return summary_df

    def create_waterfall_visualizations(self) -> plt.Figure:
        """Create comprehensive waterfall analysis visualizations"""
        df = pd.DataFrame(self.waterfall_results)
        if df.empty:
            print("No data available for visualization")
            return None
        
        fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(20, 16))
        fig.suptitle('Waterfall Analysis - Tranche-Level Performance', fontsize=16, fontweight='bold')
        
        # Colors for tranches
        tranche_colors = {
            'Senior_Debt': '#1f77b4',  # Blue
            'Sub_Debt': '#ff7f0e',     # Orange
            'Equity': '#2ca02c'        # Green
        }
        
        # Plot 1: IRR vs Required Return by Tranche
        successful_df = df[df['tranche_irr'].notna()].copy()
        if not successful_df.empty:
            for tranche in tranche_colors.keys():
                tranche_data = successful_df[successful_df['tranche'] == tranche]
                if not tranche_data.empty:
                    ax1.scatter(tranche_data['required_return'] * 100,
                              tranche_data['tranche_irr'] * 100,
                              c=tranche_colors[tranche], label=tranche, alpha=0.7, s=50)
            
            # Add diagonal line (IRR = Required Return)
            max_val = max(successful_df['required_return'].max() * 100,
                         successful_df['tranche_irr'].max() * 100) if not successful_df.empty else 15
            ax1.plot([0, max_val], [0, max_val], 'k--', alpha=0.5, label='Break-even')
        
        ax1.set_xlabel('Required Return (%)')
        ax1.set_ylabel('Tranche IRR (%)')
        ax1.set_title('Tranche IRR vs Required Return', fontweight='bold')
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        
        # Plot 2: Version Comparison (Version 2 - Version 1)
        version_comparison = []
        for country in df['country_name'].unique():
            for fund in df['fund_type'].unique():
                for pricing in df['pricing_methodology'].unique():
                    for tranche in tranche_colors.keys():
                        v1_data = df[(df['country_name'] == country) &
                                   (df['fund_type'] == fund) &
                                   (df['pricing_methodology'] == pricing) &
                                   (df['tranche'] == tranche) &
                                   (df['waterfall_version'] == 'Version_1')]
                        v2_data = df[(df['country_name'] == country) &
                                   (df['fund_type'] == fund) &
                                   (df['pricing_methodology'] == pricing) &
                                   (df['tranche'] == tranche) &
                                   (df['waterfall_version'] == 'Version_2')]
                        
                        if len(v1_data) > 0 and len(v2_data) > 0:
                            v1_irr = v1_data['tranche_irr'].iloc[0]
                            v2_irr = v2_data['tranche_irr'].iloc[0]
                            if v1_irr is not None and v2_irr is not None:
                                version_comparison.append({
                                    'country': country,
                                    'fund': fund,
                                    'pricing': pricing,
                                    'tranche': tranche,
                                    'irr_difference': (v2_irr - v1_irr) * 100
                                })
        
        if version_comparison:
            comp_df = pd.DataFrame(version_comparison)
            for tranche in tranche_colors.keys():
                tranche_data = comp_df[comp_df['tranche'] == tranche]
                if not tranche_data.empty:
                    ax2.hist(tranche_data['irr_difference'], bins=15, alpha=0.6,
                            label=tranche, color=tranche_colors[tranche])
        
        ax2.axvline(x=0, color='black', linestyle='--', alpha=0.7)
        ax2.set_xlabel('IRR Difference: Version 2 - Version 1 (%)')
        ax2.set_ylabel('Frequency')
        ax2.set_title('Impact of Retrospective Adjustment', fontweight='bold')
        ax2.legend()
        ax2.grid(True, alpha=0.3)
        
        # Plot 3: Attractive Investments by Tranche
        attractive_summary = df[df['tranche_irr'].notna()].groupby(['tranche', 'waterfall_version']).agg({
            'attractive_investment': 'sum',
            'country_name': 'count'
        }).reset_index()
        attractive_summary['success_rate'] = attractive_summary['attractive_investment'] / attractive_summary['country_name'] * 100
        
        bar_width = 0.35
        tranches = list(tranche_colors.keys())
        x_pos = np.arange(len(tranches))
        
        for i, version in enumerate(['Version_1', 'Version_2']):
            version_data = attractive_summary[attractive_summary['waterfall_version'] == version]
            success_rates = []
            for tranche in tranches:
                tranche_data = version_data[version_data['tranche'] == tranche]
                rate = tranche_data['success_rate'].iloc[0] if len(tranche_data) > 0 else 0
                success_rates.append(rate)
            
            ax3.bar(x_pos + i * bar_width, success_rates, bar_width,
                   label=version, alpha=0.8)
        
        ax3.set_xlabel('Tranche')
        ax3.set_ylabel('Success Rate (%)')
        ax3.set_title('Attractive Investments by Tranche\n(IRR > Required Return)', fontweight='bold')
        ax3.set_xticks(x_pos + bar_width / 2)
        ax3.set_xticklabels(tranches)
        ax3.legend()
        ax3.grid(True, alpha=0.3)
        
        # Plot 4: Example Time Series (Pick one scenario)
        example_scenario = df[df['tranche_irr'].notna()].iloc[0] if not df[df['tranche_irr'].notna()].empty else None
        if example_scenario is not None:
            example_country = example_scenario['country_name']
            example_fund = example_scenario['fund_type']
            example_pricing = example_scenario['pricing_methodology']
            
            # Get Version 1 and Version 2 data for this scenario
            scenario_data = df[(df['country_name'] == example_country) &
                             (df['fund_type'] == example_fund) &
                             (df['pricing_methodology'] == example_pricing)]
            
            for version in ['Version_1', 'Version_2']:
                version_data = scenario_data[scenario_data['waterfall_version'] == version]
                for tranche in tranche_colors.keys():
                    tranche_data = version_data[version_data['tranche'] == tranche]
                    if not tranche_data.empty and len(tranche_data['annual_payments'].iloc[0]) > 0:
                        payments = tranche_data['annual_payments'].iloc[0]
                        years = tranche_data['payment_years'].iloc[0]
                        linestyle = '-' if version == 'Version_1' else '--'
                        alpha = 0.8 if version == 'Version_1' else 0.6
                        ax4.plot(years, np.array(payments) / 1e6,
                               color=tranche_colors[tranche],
                               linestyle=linestyle, alpha=alpha,
                               label=f'{tranche} ({version})' if tranche == 'Senior_Debt' else "")
        
        ax4.set_xlabel('Year')
        ax4.set_ylabel('Annual Payments ($ Millions)')
        ax4.set_title(f'Payment Timeline Example\n{example_country}-{example_fund}', fontweight='bold')
        ax4.legend()
        ax4.grid(True, alpha=0.3)
        
        plt.tight_layout()
        return fig

    def get_performance_summary(self) -> Dict:
        """Generate performance summary statistics"""
        df = pd.DataFrame(self.waterfall_results)
        if df.empty:
            return {}
        
        successful_df = df[df['tranche_irr'].notna()]
        
        summary = {
            'total_scenarios': len(df),
            'successful_calculations': len(successful_df),
            'success_rate': f"{len(successful_df)/len(df)*100:.1f}%"
        }
        
        if not successful_df.empty:
            # Overall attractive investment rate
            attractive_count = successful_df['attractive_investment'].sum()
            summary['attractive_investments'] = attractive_count
            summary['attractive_rate'] = f"{attractive_count/len(successful_df)*100:.1f}%"
            
            # By tranche analysis
            tranche_summary = {}
            for tranche in ['Senior_Debt', 'Sub_Debt', 'Equity']:
                tranche_data = successful_df[successful_df['tranche'] == tranche]
                if not tranche_data.empty:
                    attractive_tranche = tranche_data['attractive_investment'].sum()
                    tranche_summary[tranche] = {
                        'scenarios': len(tranche_data),
                        'attractive': attractive_tranche,
                        'attractive_rate': f"{attractive_tranche/len(tranche_data)*100:.1f}%",
                        'avg_irr': f"{tranche_data['tranche_irr'].mean()*100:.2f}%",
                        'avg_required': f"{tranche_data['required_return'].mean()*100:.2f}%"
                    }
            
            summary['by_tranche'] = tranche_summary
            
            # Version comparison
            v1_attractive = successful_df[successful_df['waterfall_version'] == 'Version_1']['attractive_investment'].sum()
            v2_attractive = successful_df[successful_df['waterfall_version'] == 'Version_2']['attractive_investment'].sum()
            v1_total = len(successful_df[successful_df['waterfall_version'] == 'Version_1'])
            v2_total = len(successful_df[successful_df['waterfall_version'] == 'Version_2'])
            
            summary['version_comparison'] = {
                'Version_1': {
                    'attractive': v1_attractive,
                    'total': v1_total,
                    'rate': f"{v1_attractive/v1_total*100:.1f}%" if v1_total > 0 else "N/A"
                },
                'Version_2': {
                    'attractive': v2_attractive,
                    'total': v2_total,
                    'rate': f"{v2_attractive/v2_total*100:.1f}%" if v2_total > 0 else "N/A"
                }
            }
        
        return summary


def run_module_c_waterfall_analysis(module_b_results: list, aug22_csv_path: str = 'Aug22.csv'):
    """Execute Module C Waterfall Analysis"""
    print("="*80)
    print("MODULE C: WATERFALL ANALYSIS")
    print("Tranche-level cash flow distribution and IRR calculation")
    print("="*80)
    
    # Initialize waterfall analyzer
    waterfall = WaterfallAnalyzer(module_b_results, aug22_csv_path)
    
    # Process all scenarios
    print(f"\nProcessing {len(module_b_results)} scenarios...")
    waterfall_results = waterfall.process_all_scenarios()
    
    # Create and display summary table
    summary_table = waterfall.create_summary_table()
    print(f"\nWATERFALL RESULTS SUMMARY ({len(summary_table)} total results):")
    print("="*120)
    
    # Display settings for full table
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    print(summary_table.to_string(index=False))
    
    # Performance summary
    performance = waterfall.get_performance_summary()
    print(f"\n{'='*80}")
    print("WATERFALL PERFORMANCE SUMMARY:")
    print(f"{'='*80}")
    
    for key, value in performance.items():
        if key == 'by_tranche':
            print(f"\nTranche Analysis:")
            for tranche, stats in value.items():
                print(f"  {tranche}:")
                for stat_key, stat_value in stats.items():
                    print(f"    {stat_key}: {stat_value}")
        elif key == 'version_comparison':
            print(f"\nVersion Comparison:")
            for version, stats in value.items():
                print(f"  {version}: {stats['attractive']}/{stats['total']} attractive ({stats['rate']})")
        else:
            print(f"{key}: {value}")
    
    # Create visualizations
    print(f"\nGenerating waterfall visualizations...")
    fig = waterfall.create_waterfall_visualizations()
    if fig:
        plt.show()
        print("Visualizations displayed successfully")
    
    print(f"\nModule C Complete!")
    print(f"Generated {len(waterfall_results)} tranche-level results")
    print("Ready for final analysis and reporting")
    
    return waterfall_results, waterfall

In [ ]:
# FAST MEETING PLOT
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

def create_fast_meeting_plots(module_b_results):
    """Create plots for your meeting"""
    
    print("CREATING FAST MEETING PLOTS...")
    
    # Extract data with NoneType protection
    countries = []
    irrs = []
    fund_types = []
    
    for result in module_b_results:
        country = result.get('country_name')
        irr = result.get('project_irr')
        fund = result.get('fund_type', 'Unknown')
        
        # Skip NoneType and NaN values
        if country and irr is not None and not pd.isna(irr):
            countries.append(country)
            irrs.append(irr * 100)  # Convert to percentage
            fund_types.append(fund)
    
    # Create DataFrame for easier handling
    df = pd.DataFrame({
        'country': countries,
        'irr': irrs,
        'fund': fund_types
    })
    
    # Remove duplicates and take weighted_avg only
    weighted_avg_data = []
    for result in module_b_results:
        if (result.get('pricing_methodology') == 'weighted_avg' and 
            result.get('project_irr') is not None and 
            not pd.isna(result.get('project_irr'))):
            weighted_avg_data.append({
                'country': result['country_name'],
                'irr': result['project_irr'] * 100,
                'fund': result['fund_type']
            })
    
    df_weighted = pd.DataFrame(weighted_avg_data)
    
    # PLOT 1: TOP 20 COUNTRIES BY IRR
    plt.figure(figsize=(16, 10))
    
    # Get top 20 Fund B scenarios
    fund_b_data = df_weighted[df_weighted['fund'] == 'Fund B'].copy()
    fund_b_data = fund_b_data.sort_values('irr', ascending=False).head(20)
    
    plt.subplot(2, 1, 1)
    bars = plt.bar(range(len(fund_b_data)), fund_b_data['irr'], 
                   color='skyblue', alpha=0.8)
    
    # Add hurdle rate lines
    plt.axhline(y=4, color='blue', linestyle='--', alpha=0.8, label='Senior Debt Hurdle (4%)')
    plt.axhline(y=8, color='orange', linestyle='--', alpha=0.8, label='Sub Debt Hurdle (8%)')
    plt.axhline(y=12, color='red', linestyle='--', alpha=0.8, label='Equity Hurdle (12%)')
    
    plt.title('Top 20 Countries: Renewable Energy IRR (Fund B - 25 Year)', fontsize=16, fontweight='bold')
    plt.ylabel('IRR (%)', fontsize=12)
    plt.xticks(range(len(fund_b_data)), fund_b_data['country'], rotation=45, ha='right')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Add value labels on bars
    for i, (idx, row) in enumerate(fund_b_data.iterrows()):
        plt.text(i, row['irr'] + 0.2, f"{row['irr']:.1f}%", 
                ha='center', va='bottom', fontsize=8)
    
    # PLOT 2: FUND A vs FUND B COMPARISON
    plt.subplot(2, 1, 2)
    
    # Get countries that appear in both funds
    fund_a_data = df_weighted[df_weighted['fund'] == 'Fund A'].copy()
    fund_b_data = df_weighted[df_weighted['fund'] == 'Fund B'].copy()
    
    common_countries = set(fund_a_data['country']) & set(fund_b_data['country'])
    common_countries = list(common_countries)[:15]  # Top 15 for visibility
    
    fund_a_irrs = []
    fund_b_irrs = []
    
    for country in common_countries:
        a_irr = fund_a_data[fund_a_data['country'] == country]['irr'].iloc[0]
        b_irr = fund_b_data[fund_b_data['country'] == country]['irr'].iloc[0]
        fund_a_irrs.append(a_irr)
        fund_b_irrs.append(b_irr)
    
    x_pos = np.arange(len(common_countries))
    width = 0.35
    
    plt.bar(x_pos - width/2, fund_a_irrs, width, label='Fund A (10-year)', alpha=0.8, color='lightcoral')
    plt.bar(x_pos + width/2, fund_b_irrs, width, label='Fund B (25-year)', alpha=0.8, color='skyblue')
    
    # Add hurdle rate lines
    plt.axhline(y=4, color='blue', linestyle='--', alpha=0.8)
    plt.axhline(y=8, color='orange', linestyle='--', alpha=0.8)
    plt.axhline(y=12, color='red', linestyle='--', alpha=0.8)
    
    plt.title('Fund A vs Fund B: IRR Comparison (Top 15 Countries)', fontsize=16, fontweight='bold')
    plt.ylabel('IRR (%)', fontsize=12)
    plt.xlabel('Countries', fontsize=12)
    plt.xticks(x_pos, common_countries, rotation=45, ha='right')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('MEETING_PLOTS_FAST.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    # SUMMARY STATISTICS FOR MEETING
    print("\\n" + "="*80)
    print("MEETING SUMMARY STATISTICS")
    print("="*80)
    
    all_irrs = [irr for irr in df_weighted['irr'] if not pd.isna(irr)]
    
    print(f"Total scenarios analyzed: {len(module_b_results)}")
    print(f"Countries with viable IRR data: {len(set(df_weighted['country']))}")
    print(f"Average IRR across all projects: {np.mean(all_irrs):.2f}%")
    print(f"Median IRR: {np.median(all_irrs):.2f}%")
    print(f"IRR Range: {np.min(all_irrs):.2f}% to {np.max(all_irrs):.2f}%")
    
    # Countries above hurdle rates
    above_senior = len([irr for irr in all_irrs if irr > 4])
    above_sub = len([irr for irr in all_irrs if irr > 8])
    above_equity = len([irr for irr in all_irrs if irr > 12])
    
    print(f"\\nScenarios above hurdle rates:")
    print(f"  Senior Debt (>4%): {above_senior}/{len(all_irrs)} ({above_senior/len(all_irrs)*100:.1f}%)")
    print(f"  Sub Debt (>8%): {above_sub}/{len(all_irrs)} ({above_sub/len(all_irrs)*100:.1f}%)")
    print(f"  Equity (>12%): {above_equity}/{len(all_irrs)} ({above_equity/len(all_irrs)*100:.1f}%)")
    
    # Top 5 performers
    top_5 = df_weighted.nlargest(5, 'irr')
    print(f"\\nTop 5 Investment Opportunities:")
    for idx, row in top_5.iterrows():
        print(f"  {row['country']} ({row['fund']}): {row['irr']:.2f}%")
    
    print("\\n FAST PLOTS CREATED: MEETING_PLOTS_FAST.png")

# RUN THIS
create_fast_meeting_plots(module_b_results)